On this notebook I am going to take the information from the API of starwars. Then I am going to clean it and transform it into structured data.

It has no bussiness objective. It is just to ilustrate how an ETL proccess could be done but for simplicity purpouses we skip some of the steps (or at least the explanation) when they become too repetitive. Also, there are sometimes that a solution is taken and other when I decided to take another path.

Finally it's important to notice that relational databases admit now json format in some columns, so some transformations when the relationships are many-to-many are not neccesaries.

In [202]:
#Import the neccesaries libraries
import requests
import pandas as pd
import numpy as np

In [2]:
#We pull the data from the API and we examinate what it contains
api = requests.get("https://swapi.dev/api/")
api.content

b'{"people":"https://swapi.dev/api/people/","planets":"https://swapi.dev/api/planets/","films":"https://swapi.dev/api/films/","species":"https://swapi.dev/api/species/","vehicles":"https://swapi.dev/api/vehicles/","starships":"https://swapi.dev/api/starships/"}'

We have 6 different datasets. People (characters), planets, films, species, vehicles and starships.

In [3]:
#We start with "people". We pull the data from the API and we examinate it.
people = requests.get("https://swapi.dev/api/people/")
people.content

b'{"count":82,"next":"https://swapi.dev/api/people/?page=2","previous":null,"results":[{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/14/","https://swapi.dev/api/vehicles/30/"],"starships":["https://swapi.dev/api/starships/12/","https://swapi.dev/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.dev/api/people/1/"},{"name":"C-3PO","height":"167","mass":"75","hair_color":"n/a","skin_color":"gold","eye_color":"yellow","birth_year":"112BBY","gender":"n/a","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","ht

In [4]:
#We examinate it in json format (neater)
people.json()

{'count': 82,
 'next': 'https://swapi.dev/api/people/?page=2',
 'previous': None,
 'results': [{'name': 'Luke Skywalker',
   'height': '172',
   'mass': '77',
   'hair_color': 'blond',
   'skin_color': 'fair',
   'eye_color': 'blue',
   'birth_year': '19BBY',
   'gender': 'male',
   'homeworld': 'https://swapi.dev/api/planets/1/',
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/6/'],
   'species': [],
   'vehicles': ['https://swapi.dev/api/vehicles/14/',
    'https://swapi.dev/api/vehicles/30/'],
   'starships': ['https://swapi.dev/api/starships/12/',
    'https://swapi.dev/api/starships/22/'],
   'created': '2014-12-09T13:50:51.644000Z',
   'edited': '2014-12-20T21:17:56.891000Z',
   'url': 'https://swapi.dev/api/people/1/'},
  {'name': 'C-3PO',
   'height': '167',
   'mass': '75',
   'hair_color': 'n/a',
   'skin_color': 'gold',
   'eye_color': 'yellow',
   'birth_year': '112BB

First we see that we have a couple of labels of metadata in the json ('count', 'next' and 'previous' that we are not interested in). Under the label 'results' we find a list of jsons that is the data that we are interested in. Then we see that there are other pages that complement this one and that they are 82 characters registered.

In [5]:
people.json()['results']

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

By the url of the last register ('name': 'C-3PO') we can infer that there are 10 register on each page. So we can assume that they are 9 pages of people.

In [6]:
#We save this information in a dataframe of Pandas.
people_df = pd.DataFrame(people.json()['results'])
people_df

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
5,Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:52:14.024000Z,2014-12-20T21:17:50.317000Z,https://swapi.dev/api/people/6/
6,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:53:41.121000Z,2014-12-20T21:17:50.319000Z,https://swapi.dev/api/people/7/
7,R5-D4,97,32,n/a,"white, red",red,unknown,n/a,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:57:50.959000Z,2014-12-20T21:17:50.321000Z,https://swapi.dev/api/people/8/
8,Biggs Darklighter,183,84,black,light,brown,24BBY,male,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[],[],[https://swapi.dev/api/starships/12/],2014-12-10T15:59:50.509000Z,2014-12-20T21:17:50.323000Z,https://swapi.dev/api/people/9/
9,Obi-Wan Kenobi,182,77,"auburn, white",fair,blue-gray,57BBY,male,https://swapi.dev/api/planets/20/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/38/],"[https://swapi.dev/api/starships/48/, https://...",2014-12-10T16:16:29.192000Z,2014-12-20T21:17:50.325000Z,https://swapi.dev/api/people/10/


In [7]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        10 non-null     object
 1   height      10 non-null     object
 2   mass        10 non-null     object
 3   hair_color  10 non-null     object
 4   skin_color  10 non-null     object
 5   eye_color   10 non-null     object
 6   birth_year  10 non-null     object
 7   gender      10 non-null     object
 8   homeworld   10 non-null     object
 9   films       10 non-null     object
 10  species     10 non-null     object
 11  vehicles    10 non-null     object
 12  starships   10 non-null     object
 13  created     10 non-null     object
 14  edited      10 non-null     object
 15  url         10 non-null     object
dtypes: object(16)
memory usage: 1.4+ KB


We see that there are a couple of fields that we are going to need to work on them. There are lists of url that relate this table to another and there are numbers (height, mass) that are registered as objects. We are going to do these later when we have all the information.

In [8]:
#We examinate the second page.
people = requests.get("https://swapi.dev/api/people/?page=2")
people.json()

{'count': 82,
 'next': 'https://swapi.dev/api/people/?page=3',
 'previous': 'https://swapi.dev/api/people/?page=1',
 'results': [{'name': 'Anakin Skywalker',
   'height': '188',
   'mass': '84',
   'hair_color': 'blond',
   'skin_color': 'fair',
   'eye_color': 'blue',
   'birth_year': '41.9BBY',
   'gender': 'male',
   'homeworld': 'https://swapi.dev/api/planets/1/',
   'films': ['https://swapi.dev/api/films/4/',
    'https://swapi.dev/api/films/5/',
    'https://swapi.dev/api/films/6/'],
   'species': [],
   'vehicles': ['https://swapi.dev/api/vehicles/44/',
    'https://swapi.dev/api/vehicles/46/'],
   'starships': ['https://swapi.dev/api/starships/39/',
    'https://swapi.dev/api/starships/59/',
    'https://swapi.dev/api/starships/65/'],
   'created': '2014-12-10T16:20:44.310000Z',
   'edited': '2014-12-20T21:17:50.327000Z',
   'url': 'https://swapi.dev/api/people/11/'},
  {'name': 'Wilhuff Tarkin',
   'height': '180',
   'mass': 'unknown',
   'hair_color': 'auburn, grey',
   'ski

It seemed that it only changes the last number of the url (2,3,..,9). We can automatize the process.

In [9]:
for i in range(2,10):
    people = requests.get("https://swapi.dev/api/people/?page="+str(i))
    people_df = people_df.append(pd.DataFrame(people.json()['results']),ignore_index=True)

C:\Users\larra\AppData\Local\Temp\ipykernel_5380\103946009.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_df = people_df.append(pd.DataFrame(people.json()['results']),ignore_index=True)
C:\Users\larra\AppData\Local\Temp\ipykernel_5380\103946009.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_df = people_df.append(pd.DataFrame(people.json()['results']),ignore_index=True)
C:\Users\larra\AppData\Local\Temp\ipykernel_5380\103946009.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_df = people_df.append(pd.DataFrame(people.json()['results']),ignore_index=True)
C:\Users\larra\AppData\Local\Temp\ipykernel_5380\103946009.py:3: FutureWarning: The frame.append method is deprecated and will be removed f

In [10]:
#We inspect the dataframe and we see that we have the 82 registers.
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: object(16)
memory usage: 10.4+ KB


We need to work on it but first we are going to pull the rest of the datasets

In [11]:
#We continue with planets dataset.
planets = requests.get("https://swapi.dev/api/planets/")
planets.json()

{'count': 60,
 'next': 'https://swapi.dev/api/planets/?page=2',
 'previous': None,
 'results': [{'name': 'Tatooine',
   'rotation_period': '23',
   'orbital_period': '304',
   'diameter': '10465',
   'climate': 'arid',
   'gravity': '1 standard',
   'terrain': 'desert',
   'surface_water': '1',
   'population': '200000',
   'residents': ['https://swapi.dev/api/people/1/',
    'https://swapi.dev/api/people/2/',
    'https://swapi.dev/api/people/4/',
    'https://swapi.dev/api/people/6/',
    'https://swapi.dev/api/people/7/',
    'https://swapi.dev/api/people/8/',
    'https://swapi.dev/api/people/9/',
    'https://swapi.dev/api/people/11/',
    'https://swapi.dev/api/people/43/',
    'https://swapi.dev/api/people/62/'],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/4/',
    'https://swapi.dev/api/films/5/',
    'https://swapi.dev/api/films/6/'],
   'created': '2014-12-09T13:50:49.641000Z',
   'edited': '2014-12-20T

In [12]:
#They are 60 planets so we are going to pull 6 pages. First we make the first dataframe with the datasets that we already pull.
planets_df = pd.DataFrame(planets.json()['results'])
planets_df

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/
5,Bespin,12,5110,118000,temperate,"1.5 (surface), 1 standard (Cloud City)",gas giant,0,6000000,[https://swapi.dev/api/people/26/],[https://swapi.dev/api/films/2/],2014-12-10T11:43:55.240000Z,2014-12-20T20:58:18.427000Z,https://swapi.dev/api/planets/6/
6,Endor,18,402,4900,temperate,0.85 standard,"forests, mountains, lakes",8,30000000,[https://swapi.dev/api/people/30/],[https://swapi.dev/api/films/3/],2014-12-10T11:50:29.349000Z,2014-12-20T20:58:18.429000Z,https://swapi.dev/api/planets/7/
7,Naboo,26,312,12120,temperate,1 standard,"grassy hills, swamps, forests, mountains",12,4500000000,"[https://swapi.dev/api/people/3/, https://swap...","[https://swapi.dev/api/films/3/, https://swapi...",2014-12-10T11:52:31.066000Z,2014-12-20T20:58:18.430000Z,https://swapi.dev/api/planets/8/
8,Coruscant,24,368,12240,temperate,1 standard,"cityscape, mountains",unknown,1000000000000,"[https://swapi.dev/api/people/34/, https://swa...","[https://swapi.dev/api/films/3/, https://swapi...",2014-12-10T11:54:13.921000Z,2014-12-20T20:58:18.432000Z,https://swapi.dev/api/planets/9/
9,Kamino,27,463,19720,temperate,1 standard,ocean,100,1000000000,"[https://swapi.dev/api/people/22/, https://swa...",[https://swapi.dev/api/films/5/],2014-12-10T12:45:06.577000Z,2014-12-20T20:58:18.434000Z,https://swapi.dev/api/planets/10/


In [13]:
for i in range(2,7):
    planets = requests.get("https://swapi.dev/api/planets/?page="+str(i))
    planets_df = pd.concat([planets_df,pd.DataFrame(planets.json()['results'])],ignore_index=True)

In [14]:
planets_df.head()

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/


In [15]:
#We continue with films
films = requests.get("https://swapi.dev/api/films/")
films.json()

{'count': 6,
 'next': None,
 'previous': None,
 'results': [{'title': 'A New Hope',
   'episode_id': 4,
   'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....",
   'director': 'George Lucas',
   'producer': 'Gary Kurtz, Rick McCallum',
   'release_date': '1977-05-25',
   'characters': ['https://swapi.dev/api/people/1/',
    'https://swapi.dev/api/people/2/',
    'https://swapi.dev/api/people/3/',
    'https://swapi.dev/api/people/4/',
    'https://swapi.dev/api/peopl

In [16]:
#Just one page so there is no need of more pulls.
films_df = pd.DataFrame(films.json()['results'])
films_df

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/4/, https://swa...","[https://swapi.dev/api/starships/3/, https://s...","[https://swapi.dev/api/vehicles/8/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/8/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-18T10:39:33.255000Z,2014-12-20T09:48:37.462000Z,https://swapi.dev/api/films/3/
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/31/, https://...","[https://swapi.dev/api/vehicles/33/, https://s...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-19T16:52:55.740000Z,2014-12-20T10:54:07.216000Z,https://swapi.dev/api/films/4/
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/21/, https://...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-20T10:57:57.886000Z,2014-12-20T20:18:48.516000Z,https://swapi.dev/api/films/5/
5,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/33/, https://s...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-20T18:49:38.403000Z,2014-12-20T20:47:52.073000Z,https://swapi.dev/api/films/6/


In [17]:
#We continue with species.
species = requests.get("https://swapi.dev/api/species/")
species.json()

{'count': 37,
 'next': 'https://swapi.dev/api/species/?page=2',
 'previous': None,
 'results': [{'name': 'Human',
   'classification': 'mammal',
   'designation': 'sentient',
   'average_height': '180',
   'skin_colors': 'caucasian, black, asian, hispanic',
   'hair_colors': 'blonde, brown, black, red',
   'eye_colors': 'brown, blue, green, hazel, grey, amber',
   'average_lifespan': '120',
   'homeworld': 'https://swapi.dev/api/planets/9/',
   'language': 'Galactic Basic',
   'people': ['https://swapi.dev/api/people/66/',
    'https://swapi.dev/api/people/67/',
    'https://swapi.dev/api/people/68/',
    'https://swapi.dev/api/people/74/'],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/4/',
    'https://swapi.dev/api/films/5/',
    'https://swapi.dev/api/films/6/'],
   'created': '2014-12-10T13:52:11.567000Z',
   'edited': '2014-12-20T21:36:42.136000Z',
   'url': 'https://swap

In [18]:
#'count' = 37. 4 pages then.
species_df = pd.DataFrame(species.json()['results'])
for i in range(2,5):
    species = requests.get("https://swapi.dev/api/species/?page="+str(i))
    species_df = pd.concat([species_df,pd.DataFrame(species.json()['results'])],ignore_index=True)

In [19]:
species_df.head()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/


In [20]:
species_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              37 non-null     object
 1   classification    37 non-null     object
 2   designation       37 non-null     object
 3   average_height    37 non-null     object
 4   skin_colors       37 non-null     object
 5   hair_colors       37 non-null     object
 6   eye_colors        37 non-null     object
 7   average_lifespan  37 non-null     object
 8   homeworld         36 non-null     object
 9   language          37 non-null     object
 10  people            37 non-null     object
 11  films             37 non-null     object
 12  created           37 non-null     object
 13  edited            37 non-null     object
 14  url               37 non-null     object
dtypes: object(15)
memory usage: 4.5+ KB


In [21]:
#We continue with vehicles.
vehicles = requests.get("https://swapi.dev/api/vehicles/")
vehicles.json()

{'count': 39,
 'next': 'https://swapi.dev/api/vehicles/?page=2',
 'previous': None,
 'results': [{'name': 'Sand Crawler',
   'model': 'Digger Crawler',
   'manufacturer': 'Corellia Mining Corporation',
   'cost_in_credits': '150000',
   'length': '36.8 ',
   'max_atmosphering_speed': '30',
   'crew': '46',
   'passengers': '30',
   'cargo_capacity': '50000',
   'consumables': '2 months',
   'vehicle_class': 'wheeled',
   'pilots': [],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/5/'],
   'created': '2014-12-10T15:36:25.724000Z',
   'edited': '2014-12-20T21:30:21.661000Z',
   'url': 'https://swapi.dev/api/vehicles/4/'},
  {'name': 'T-16 skyhopper',
   'model': 'T-16 skyhopper',
   'manufacturer': 'Incom Corporation',
   'cost_in_credits': '14500',
   'length': '10.4 ',
   'max_atmosphering_speed': '1200',
   'crew': '1',
   'passengers': '1',
   'cargo_capacity': '50',
   'consumables': '0',
   'vehicle_class': 'repulsorcraft',
   'pilots': [],
   'fil

In [22]:
#'count' = 39. 4 pages then.
vehicles_df = pd.DataFrame(vehicles.json()['results'])
for i in range(2,5):
    vehicles = requests.get("https://swapi.dev/api/vehicles/?page="+str(i))
    vehicles_df = pd.concat([vehicles_df,pd.DataFrame(vehicles.json()['results'])],ignore_index=True)

In [23]:
vehicles_df.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,https://swapi.dev/api/vehicles/4/
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:01:52.434000Z,2014-12-20T21:30:21.665000Z,https://swapi.dev/api/vehicles/6/
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:13:52.586000Z,2014-12-20T21:30:21.668000Z,https://swapi.dev/api/vehicles/7/
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:33:52.860000Z,2014-12-20T21:30:21.670000Z,https://swapi.dev/api/vehicles/8/
4,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"[https://swapi.dev/api/people/1/, https://swap...",[https://swapi.dev/api/films/2/],2014-12-15T12:22:12Z,2014-12-20T21:30:21.672000Z,https://swapi.dev/api/vehicles/14/


In [24]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    39 non-null     object
 1   model                   39 non-null     object
 2   manufacturer            39 non-null     object
 3   cost_in_credits         39 non-null     object
 4   length                  39 non-null     object
 5   max_atmosphering_speed  39 non-null     object
 6   crew                    39 non-null     object
 7   passengers              39 non-null     object
 8   cargo_capacity          39 non-null     object
 9   consumables             39 non-null     object
 10  vehicle_class           39 non-null     object
 11  pilots                  39 non-null     object
 12  films                   39 non-null     object
 13  created                 39 non-null     object
 14  edited                  39 non-null     object
 15  url     

In [25]:
#We finish with starships
starships = requests.get("https://swapi.dev/api/starships/")
starships.json()

{'count': 36,
 'next': 'https://swapi.dev/api/starships/?page=2',
 'previous': None,
 'results': [{'name': 'CR90 corvette',
   'model': 'CR90 corvette',
   'manufacturer': 'Corellian Engineering Corporation',
   'cost_in_credits': '3500000',
   'length': '150',
   'max_atmosphering_speed': '950',
   'crew': '30-165',
   'passengers': '600',
   'cargo_capacity': '3000000',
   'consumables': '1 year',
   'hyperdrive_rating': '2.0',
   'MGLT': '60',
   'starship_class': 'corvette',
   'pilots': [],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/6/'],
   'created': '2014-12-10T14:20:33.369000Z',
   'edited': '2014-12-20T21:23:49.867000Z',
   'url': 'https://swapi.dev/api/starships/2/'},
  {'name': 'Star Destroyer',
   'model': 'Imperial I-class Star Destroyer',
   'manufacturer': 'Kuat Drive Yards',
   'cost_in_credits': '150000000',
   'length': '1,600',
   'max_atmosphering_speed': '975',
   'crew': '47,060',
   'pass

In [26]:
#'count' = 36. 4 pages then.
starships_df = pd.DataFrame(starships.json()['results'])
for i in range(2,5):
    starships = requests.get("https://swapi.dev/api/starships/?page="+str(i))
    starships_df = pd.concat([starships_df,pd.DataFrame(starships.json()['results'])],ignore_index=True)

In [27]:
starships_df.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/


In [28]:
starships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    36 non-null     object
 1   model                   36 non-null     object
 2   manufacturer            36 non-null     object
 3   cost_in_credits         36 non-null     object
 4   length                  36 non-null     object
 5   max_atmosphering_speed  36 non-null     object
 6   crew                    36 non-null     object
 7   passengers              36 non-null     object
 8   cargo_capacity          36 non-null     object
 9   consumables             36 non-null     object
 10  hyperdrive_rating       36 non-null     object
 11  MGLT                    36 non-null     object
 12  starship_class          36 non-null     object
 13  pilots                  36 non-null     object
 14  films                   36 non-null     object
 15  created 

We need to start inspecting our dataframes in order to transofrm it so we can process them.

In [29]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: object(16)
memory usage: 10.4+ KB


In [30]:
people_df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/


In [31]:
#Height and mass should be numbers. We are going to look them closely.
people_df['height'].value_counts()

183        7
180        5
188        5
170        4
178        4
196        3
191        3
175        3
193        3
163        2
165        2
206        2
150        2
96         2
185        2
167        2
198        2
122        1
79         1
216        1
94         1
229        1
213        1
184        1
264        1
157        1
112        1
166        1
168        1
171        1
172        1
137        1
224        1
160        1
88         1
unknown    1
177        1
190        1
200        1
66         1
173        1
228        1
182        1
97         1
202        1
234        1
Name: height, dtype: int64

In [32]:
#There is just one unknown value. We see where it is.
people_df[people_df['height']=='unknown']

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
27,Arvel Crynyd,unknown,unknown,brown,fair,brown,unknown,male,https://swapi.dev/api/planets/28/,[https://swapi.dev/api/films/3/],[],[],[https://swapi.dev/api/starships/28/],2014-12-18T11:16:33.020000Z,2014-12-20T21:17:50.367000Z,https://swapi.dev/api/people/29/


In [33]:
#Is the character number 29. We are going to replace it with a 0 so we can transform this column into a whole number.
people_df['height'].replace('unknown',value=0,inplace=True)
people_df[people_df['height']==0]

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
27,Arvel Crynyd,0,unknown,brown,fair,brown,unknown,male,https://swapi.dev/api/planets/28/,[https://swapi.dev/api/films/3/],[],[],[https://swapi.dev/api/starships/28/],2014-12-18T11:16:33.020000Z,2014-12-20T21:17:50.367000Z,https://swapi.dev/api/people/29/


In [34]:
people_df = people_df.astype({'height':'int64'})
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     int64 
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: int64(1), object(15)
memory usage: 10.4+ KB


In [35]:
#We make the same thing with mass.
people_df['mass'].value_counts()

unknown    23
80          6
79          4
77          3
84          3
75          3
32          2
136         2
82          2
48          2
45          2
55          2
50          2
15          1
40          1
56.2        1
65          1
87          1
85          1
102         1
88          1
57          1
66          1
20          1
90          1
89          1
68          1
83          1
113         1
140         1
78.2        1
17          1
110         1
1,358       1
74          1
112         1
120         1
49          1
159         1
Name: mass, dtype: int64

In [36]:
#There are 23 unknown values here. We inspect them.
people_df[people_df['mass']=='unknown']

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
11,Wilhuff Tarkin,180,unknown,"auburn, grey",fair,blue,64BBY,male,https://swapi.dev/api/planets/21/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T16:26:56.138000Z,2014-12-20T21:17:50.330000Z,https://swapi.dev/api/people/12/
26,Mon Mothma,150,unknown,auburn,fair,blue,48BBY,female,https://swapi.dev/api/planets/32/,[https://swapi.dev/api/films/3/],[],[],[],2014-12-18T11:12:38.895000Z,2014-12-20T21:17:50.364000Z,https://swapi.dev/api/people/28/
27,Arvel Crynyd,0,unknown,brown,fair,brown,unknown,male,https://swapi.dev/api/planets/28/,[https://swapi.dev/api/films/3/],[],[],[https://swapi.dev/api/starships/28/],2014-12-18T11:16:33.020000Z,2014-12-20T21:17:50.367000Z,https://swapi.dev/api/people/29/
32,Finis Valorum,170,unknown,blond,fair,blue,91BBY,male,https://swapi.dev/api/planets/9/,[https://swapi.dev/api/films/4/],[],[],[],2014-12-19T17:21:45.915000Z,2014-12-20T21:17:50.379000Z,https://swapi.dev/api/people/34/
36,Rugor Nass,206,unknown,none,green,orange,unknown,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[https://swapi.dev/api/species/12/],[],[],2014-12-19T17:33:38.909000Z,2014-12-20T21:17:50.388000Z,https://swapi.dev/api/people/38/
37,Ric Olié,183,unknown,brown,fair,blue,unknown,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[],[],[https://swapi.dev/api/starships/40/],2014-12-19T17:45:01.522000Z,2014-12-20T21:17:50.392000Z,https://swapi.dev/api/people/39/
38,Watto,137,unknown,black,"blue, grey",yellow,unknown,male,https://swapi.dev/api/planets/34/,"[https://swapi.dev/api/films/4/, https://swapi...",[https://swapi.dev/api/species/13/],[],[],2014-12-19T17:48:54.647000Z,2014-12-20T21:17:50.395000Z,https://swapi.dev/api/people/40/
40,Quarsh Panaka,183,unknown,black,dark,brown,62BBY,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[],[],[],2014-12-19T17:55:43.348000Z,2014-12-20T21:17:50.399000Z,https://swapi.dev/api/people/42/
41,Shmi Skywalker,163,unknown,black,fair,brown,72BBY,female,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/4/, https://swapi...",[],[],[],2014-12-19T17:57:41.191000Z,2014-12-20T21:17:50.401000Z,https://swapi.dev/api/people/43/
43,Bib Fortuna,180,unknown,none,pale,pink,unknown,male,https://swapi.dev/api/planets/37/,[https://swapi.dev/api/films/3/],[https://swapi.dev/api/species/15/],[],[],2014-12-20T09:47:02.512000Z,2014-12-20T21:17:50.407000Z,https://swapi.dev/api/people/45/


In [37]:
#There are more than 1/4 of the values that are going to be 0. We are going to do it anyway but we should report it.
people_df['mass'].replace('unknown',value=0,inplace=True)
people_df[people_df['mass']==0]

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
11,Wilhuff Tarkin,180,0,"auburn, grey",fair,blue,64BBY,male,https://swapi.dev/api/planets/21/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T16:26:56.138000Z,2014-12-20T21:17:50.330000Z,https://swapi.dev/api/people/12/
26,Mon Mothma,150,0,auburn,fair,blue,48BBY,female,https://swapi.dev/api/planets/32/,[https://swapi.dev/api/films/3/],[],[],[],2014-12-18T11:12:38.895000Z,2014-12-20T21:17:50.364000Z,https://swapi.dev/api/people/28/
27,Arvel Crynyd,0,0,brown,fair,brown,unknown,male,https://swapi.dev/api/planets/28/,[https://swapi.dev/api/films/3/],[],[],[https://swapi.dev/api/starships/28/],2014-12-18T11:16:33.020000Z,2014-12-20T21:17:50.367000Z,https://swapi.dev/api/people/29/
32,Finis Valorum,170,0,blond,fair,blue,91BBY,male,https://swapi.dev/api/planets/9/,[https://swapi.dev/api/films/4/],[],[],[],2014-12-19T17:21:45.915000Z,2014-12-20T21:17:50.379000Z,https://swapi.dev/api/people/34/
36,Rugor Nass,206,0,none,green,orange,unknown,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[https://swapi.dev/api/species/12/],[],[],2014-12-19T17:33:38.909000Z,2014-12-20T21:17:50.388000Z,https://swapi.dev/api/people/38/
37,Ric Olié,183,0,brown,fair,blue,unknown,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[],[],[https://swapi.dev/api/starships/40/],2014-12-19T17:45:01.522000Z,2014-12-20T21:17:50.392000Z,https://swapi.dev/api/people/39/
38,Watto,137,0,black,"blue, grey",yellow,unknown,male,https://swapi.dev/api/planets/34/,"[https://swapi.dev/api/films/4/, https://swapi...",[https://swapi.dev/api/species/13/],[],[],2014-12-19T17:48:54.647000Z,2014-12-20T21:17:50.395000Z,https://swapi.dev/api/people/40/
40,Quarsh Panaka,183,0,black,dark,brown,62BBY,male,https://swapi.dev/api/planets/8/,[https://swapi.dev/api/films/4/],[],[],[],2014-12-19T17:55:43.348000Z,2014-12-20T21:17:50.399000Z,https://swapi.dev/api/people/42/
41,Shmi Skywalker,163,0,black,fair,brown,72BBY,female,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/4/, https://swapi...",[],[],[],2014-12-19T17:57:41.191000Z,2014-12-20T21:17:50.401000Z,https://swapi.dev/api/people/43/
43,Bib Fortuna,180,0,none,pale,pink,unknown,male,https://swapi.dev/api/planets/37/,[https://swapi.dev/api/films/3/],[https://swapi.dev/api/species/15/],[],[],2014-12-20T09:47:02.512000Z,2014-12-20T21:17:50.407000Z,https://swapi.dev/api/people/45/


In [38]:
#We should replace also the mass of 1,358. We inspect it first.
people_df[people_df['mass']=='1,358']

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
15,Jabba Desilijic Tiure,175,"1,358",n/a,"green-tan, brown",orange,600BBY,hermaphrodite,https://swapi.dev/api/planets/24/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/5/],[],[],2014-12-10T17:11:31.638000Z,2014-12-20T21:17:50.338000Z,https://swapi.dev/api/people/16/


In [39]:
people_df['mass'].replace('1,358',value=1358,inplace=True)
people_df[people_df['mass']==1358]

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
15,Jabba Desilijic Tiure,175,1358,n/a,"green-tan, brown",orange,600BBY,hermaphrodite,https://swapi.dev/api/planets/24/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/5/],[],[],2014-12-10T17:11:31.638000Z,2014-12-20T21:17:50.338000Z,https://swapi.dev/api/people/16/


In [40]:
#We transform then mass into float numbers (float64).
people_df = people_df.astype({'mass':'float64'})
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        82 non-null     object 
 1   height      82 non-null     int64  
 2   mass        82 non-null     float64
 3   hair_color  82 non-null     object 
 4   skin_color  82 non-null     object 
 5   eye_color   82 non-null     object 
 6   birth_year  82 non-null     object 
 7   gender      82 non-null     object 
 8   homeworld   82 non-null     object 
 9   films       82 non-null     object 
 10  species     82 non-null     object 
 11  vehicles    82 non-null     object 
 12  starships   82 non-null     object 
 13  created     82 non-null     object 
 14  edited      82 non-null     object 
 15  url         82 non-null     object 
dtypes: float64(1), int64(1), object(14)
memory usage: 10.4+ KB


In [41]:
#We are going to take a look to the first 20 rows. But we are going to drop everything between name and homeworld.
people_df.drop(columns=people_df.columns[1:8]).head(20)

,name,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
5,Owen Lars,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:52:14.024000Z,2014-12-20T21:17:50.317000Z,https://swapi.dev/api/people/6/
6,Beru Whitesun lars,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:53:41.121000Z,2014-12-20T21:17:50.319000Z,https://swapi.dev/api/people/7/
7,R5-D4,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:57:50.959000Z,2014-12-20T21:17:50.321000Z,https://swapi.dev/api/people/8/
8,Biggs Darklighter,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[],[],[https://swapi.dev/api/starships/12/],2014-12-10T15:59:50.509000Z,2014-12-20T21:17:50.323000Z,https://swapi.dev/api/people/9/
9,Obi-Wan Kenobi,https://swapi.dev/api/planets/20/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/38/],"[https://swapi.dev/api/starships/48/, https://...",2014-12-10T16:16:29.192000Z,2014-12-20T21:17:50.325000Z,https://swapi.dev/api/people/10/


In [42]:
#We notice that between the character 16 (Jabba) and the next (Wedge) is not the 17. We are going to see what happens.
people_df.tail()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
77,Grievous,216,159.0,none,"brown, white","green, yellow",unknown,male,https://swapi.dev/api/planets/59/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/36/],[https://swapi.dev/api/vehicles/60/],[https://swapi.dev/api/starships/74/],2014-12-20T19:43:53.348000Z,2014-12-20T21:17:50.488000Z,https://swapi.dev/api/people/79/
78,Tarfful,234,136.0,brown,brown,blue,unknown,male,https://swapi.dev/api/planets/14/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/3/],[],[],2014-12-20T19:46:34.209000Z,2014-12-20T21:17:50.491000Z,https://swapi.dev/api/people/80/
79,Raymus Antilles,188,79.0,brown,light,brown,unknown,male,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-20T19:49:35.583000Z,2014-12-20T21:17:50.493000Z,https://swapi.dev/api/people/81/
80,Sly Moore,178,48.0,none,pale,white,unknown,female,https://swapi.dev/api/planets/60/,"[https://swapi.dev/api/films/5/, https://swapi...",[],[],[],2014-12-20T20:18:37.619000Z,2014-12-20T21:17:50.496000Z,https://swapi.dev/api/people/82/
81,Tion Medon,206,80.0,none,grey,black,unknown,male,https://swapi.dev/api/planets/12/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/37/],[],[],2014-12-20T20:35:04.260000Z,2014-12-20T21:17:50.498000Z,https://swapi.dev/api/people/83/


In [43]:
#The last character is the character 83 so everything is correct. The API just has no character 17.
#We make a numeric Id of the characters so we can discard the last column with the URL that is useless.
#First we find the position of the number in the url chain.
people_df['url'][0].index('1')

29

In [44]:
#From the position 29 until the last character we have every number of character. We can make a list of ids.
ids = []
for i in range(people_df.shape[0]):
    ids.append(int(people_df['url'][i][29:-1]))
ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83]

In [45]:
#Now we insert the column Id in the Dataframe
people_df.insert(loc=0,column='People_Id',value=ids)
people_df

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,79,Grievous,216,159.0,none,"brown, white","green, yellow",unknown,male,https://swapi.dev/api/planets/59/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/36/],[https://swapi.dev/api/vehicles/60/],[https://swapi.dev/api/starships/74/],2014-12-20T19:43:53.348000Z,2014-12-20T21:17:50.488000Z,https://swapi.dev/api/people/79/
78,80,Tarfful,234,136.0,brown,brown,blue,unknown,male,https://swapi.dev/api/planets/14/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/3/],[],[],2014-12-20T19:46:34.209000Z,2014-12-20T21:17:50.491000Z,https://swapi.dev/api/people/80/
79,81,Raymus Antilles,188,79.0,brown,light,brown,unknown,male,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-20T19:49:35.583000Z,2014-12-20T21:17:50.493000Z,https://swapi.dev/api/people/81/
80,82,Sly Moore,178,48.0,none,pale,white,unknown,female,https://swapi.dev/api/planets/60/,"[https://swapi.dev/api/films/5/, https://swapi...",[],[],[],2014-12-20T20:18:37.619000Z,2014-12-20T21:17:50.496000Z,https://swapi.dev/api/people/82/


In [46]:
#We now inspect the column homeworlds
people_df['homeworld']

0      https://swapi.dev/api/planets/1/
1      https://swapi.dev/api/planets/1/
2      https://swapi.dev/api/planets/8/
3      https://swapi.dev/api/planets/1/
4      https://swapi.dev/api/planets/2/
                    ...                
77    https://swapi.dev/api/planets/59/
78    https://swapi.dev/api/planets/14/
79     https://swapi.dev/api/planets/2/
80    https://swapi.dev/api/planets/60/
81    https://swapi.dev/api/planets/12/
Name: homeworld, Length: 82, dtype: object

In [47]:
#Each people does not have more than homeworld. We can make an Id to identify with the df planets.
people_df['homeworld'][0].index('1')

30

In [48]:
#We make the list and then we inserted in the psoition 9 (before homeworld).
hw_ids = []
for i in range(people_df.shape[0]):
    hw_ids.append(int(people_df['homeworld'][i][30:-1]))
hw_ids

[1,
 1,
 8,
 1,
 2,
 1,
 1,
 1,
 1,
 20,
 1,
 21,
 14,
 22,
 23,
 24,
 22,
 26,
 28,
 8,
 10,
 28,
 29,
 30,
 6,
 31,
 32,
 28,
 7,
 33,
 28,
 18,
 9,
 8,
 8,
 8,
 8,
 8,
 34,
 35,
 8,
 1,
 36,
 37,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 9,
 47,
 48,
 49,
 50,
 8,
 8,
 1,
 11,
 51,
 51,
 8,
 52,
 2,
 53,
 54,
 55,
 10,
 10,
 9,
 28,
 56,
 57,
 58,
 59,
 14,
 2,
 60,
 12]

In [49]:
people_df.insert(loc=9,column='Homeworld_planetId',value=hw_ids)
people_df

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,homeworld,films,species,vehicles,starships,created,edited,url
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,79,Grievous,216,159.0,none,"brown, white","green, yellow",unknown,male,59,https://swapi.dev/api/planets/59/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/36/],[https://swapi.dev/api/vehicles/60/],[https://swapi.dev/api/starships/74/],2014-12-20T19:43:53.348000Z,2014-12-20T21:17:50.488000Z,https://swapi.dev/api/people/79/
78,80,Tarfful,234,136.0,brown,brown,blue,unknown,male,14,https://swapi.dev/api/planets/14/,[https://swapi.dev/api/films/6/],[https://swapi.dev/api/species/3/],[],[],2014-12-20T19:46:34.209000Z,2014-12-20T21:17:50.491000Z,https://swapi.dev/api/people/80/
79,81,Raymus Antilles,188,79.0,brown,light,brown,unknown,male,2,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-20T19:49:35.583000Z,2014-12-20T21:17:50.493000Z,https://swapi.dev/api/people/81/
80,82,Sly Moore,178,48.0,none,pale,white,unknown,female,60,https://swapi.dev/api/planets/60/,"[https://swapi.dev/api/films/5/, https://swapi...",[],[],[],2014-12-20T20:18:37.619000Z,2014-12-20T21:17:50.496000Z,https://swapi.dev/api/people/82/


We clean a little bit. We drop homeworld, url, created and edited (this last ones because we can make no improvement with them)

In [50]:
people_df.drop(columns=['homeworld','url','created','edited'],inplace=True)
people_df.head()

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,species,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[]


In [51]:
#We take a look into species.
people_df['species']

0                                      []
1      [https://swapi.dev/api/species/2/]
2      [https://swapi.dev/api/species/2/]
3                                      []
4                                      []
                     ...                 
77    [https://swapi.dev/api/species/36/]
78     [https://swapi.dev/api/species/3/]
79                                     []
80                                     []
81    [https://swapi.dev/api/species/37/]
Name: species, Length: 82, dtype: object

In [52]:
people_df['species'].value_counts()

[]                                     32
[https://swapi.dev/api/species/2/]      4
[https://swapi.dev/api/species/1/]      4
[https://swapi.dev/api/species/12/]     3
[https://swapi.dev/api/species/29/]     2
[https://swapi.dev/api/species/3/]      2
[https://swapi.dev/api/species/32/]     2
[https://swapi.dev/api/species/15/]     2
[https://swapi.dev/api/species/22/]     2
[https://swapi.dev/api/species/28/]     1
[https://swapi.dev/api/species/25/]     1
[https://swapi.dev/api/species/26/]     1
[https://swapi.dev/api/species/27/]     1
[https://swapi.dev/api/species/36/]     1
[https://swapi.dev/api/species/35/]     1
[https://swapi.dev/api/species/23/]     1
[https://swapi.dev/api/species/30/]     1
[https://swapi.dev/api/species/31/]     1
[https://swapi.dev/api/species/33/]     1
[https://swapi.dev/api/species/34/]     1
[https://swapi.dev/api/species/24/]     1
[https://swapi.dev/api/species/19/]     1
[https://swapi.dev/api/species/21/]     1
[https://swapi.dev/api/species/20/

In [53]:
#We have naturally 1 specie (or 0 for some) for each character. We do the same as for homeworld.
people_df['species'][1][0].index('2')

30

In [54]:
sp_ids = []
for i in range(people_df.shape[0]):
    if len(people_df['species'][i]) == 0:
        sp_ids.append(0)
    else:
        sp_ids.append(int(people_df['species'][i][0][30:-1]))
sp_ids

[0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 3,
 0,
 4,
 5,
 0,
 0,
 6,
 0,
 0,
 2,
 7,
 0,
 0,
 8,
 0,
 0,
 9,
 10,
 0,
 11,
 0,
 0,
 12,
 12,
 12,
 0,
 13,
 14,
 0,
 0,
 22,
 15,
 15,
 16,
 17,
 18,
 19,
 0,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 0,
 0,
 0,
 28,
 29,
 29,
 1,
 1,
 1,
 0,
 30,
 31,
 32,
 32,
 1,
 0,
 33,
 34,
 35,
 36,
 3,
 0,
 0,
 37]

In [55]:
#We insert it before species (column 11).
people_df.insert(loc=11,column='Species_Id',value=sp_ids)
people_df

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,species,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,[https://swapi.dev/api/species/2/],[],[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",2,[https://swapi.dev/api/species/2/],[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[],[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[https://swapi.dev/api/vehicles/30/],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,79,Grievous,216,159.0,none,"brown, white","green, yellow",unknown,male,59,[https://swapi.dev/api/films/6/],36,[https://swapi.dev/api/species/36/],[https://swapi.dev/api/vehicles/60/],[https://swapi.dev/api/starships/74/]
78,80,Tarfful,234,136.0,brown,brown,blue,unknown,male,14,[https://swapi.dev/api/films/6/],3,[https://swapi.dev/api/species/3/],[],[]
79,81,Raymus Antilles,188,79.0,brown,light,brown,unknown,male,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[],[]
80,82,Sly Moore,178,48.0,none,pale,white,unknown,female,60,"[https://swapi.dev/api/films/5/, https://swapi...",0,[],[],[]


In [56]:
people_df.drop(columns='species',inplace=True)
people_df.head()

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,[],[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",2,[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,[https://swapi.dev/api/vehicles/30/],[]


In [57]:
#We take a look into species_df
species_df.head()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/


In [58]:
#A deeper look.
species_df['people'][0]

['https://swapi.dev/api/people/66/',
 'https://swapi.dev/api/people/67/',
 'https://swapi.dev/api/people/68/',
 'https://swapi.dev/api/people/74/']

In [59]:
#We look into the people wich the species id = 1
people_df[people_df['Species_Id']==1]

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,vehicles,starships
64,66,Dormé,165,0.0,brown,light,brown,unknown,female,8,[https://swapi.dev/api/films/5/],1,[],[]
65,67,Dooku,193,80.0,white,fair,brown,102BBY,male,52,"[https://swapi.dev/api/films/5/, https://swapi...",1,[https://swapi.dev/api/vehicles/55/],[]
66,68,Bail Prestor Organa,191,0.0,black,tan,brown,67BBY,male,2,"[https://swapi.dev/api/films/5/, https://swapi...",1,[],[]
72,74,Jocasta Nu,167,0.0,white,fair,blue,unknown,female,9,[https://swapi.dev/api/films/5/],1,[],[]


In [60]:
#It is coherent what the species_df informs. I dont know why Luke or Leia are not consider human but we are going to leave it.
#The other columns we are going to leave it like this for the moment.
#We take a look into planets
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             60 non-null     object
 1   rotation_period  60 non-null     object
 2   orbital_period   60 non-null     object
 3   diameter         60 non-null     object
 4   climate          60 non-null     object
 5   gravity          60 non-null     object
 6   terrain          60 non-null     object
 7   surface_water    60 non-null     object
 8   population       60 non-null     object
 9   residents        60 non-null     object
 10  films            60 non-null     object
 11  created          60 non-null     object
 12  edited           60 non-null     object
 13  url              60 non-null     object
dtypes: object(14)
memory usage: 6.7+ KB


In [61]:
planets_df.head()

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/


In [62]:
#We are going to start defining the Id
planets_df['url'][0].index('1')

30

In [63]:
pl_ids = []
for i in range(planets_df.shape[0]):
    pl_ids.append(int(planets_df['url'][i][30:-1]))
pl_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [64]:
#We insert it as the first column
planets_df.insert(loc=0,column='Planet_Id',value=pl_ids)
planets_df. head()

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,1,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,2,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,3,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/
3,4,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/
4,5,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/


In [65]:
#We are going to check the residents of the first planet
planets_df['residents'][0]

['https://swapi.dev/api/people/1/',
 'https://swapi.dev/api/people/2/',
 'https://swapi.dev/api/people/4/',
 'https://swapi.dev/api/people/6/',
 'https://swapi.dev/api/people/7/',
 'https://swapi.dev/api/people/8/',
 'https://swapi.dev/api/people/9/',
 'https://swapi.dev/api/people/11/',
 'https://swapi.dev/api/people/43/',
 'https://swapi.dev/api/people/62/']

In [66]:
#Now we check in people_df the ones that belongs to planet 1
people_df[people_df['Homeworld_planetId']==1]

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[https://swapi.dev/api/starships/13/]
5,6,Owen Lars,178,120.0,"brown, grey",light,blue,52BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[]
6,7,Beru Whitesun lars,165,75.0,brown,light,blue,47BBY,female,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[]
7,8,R5-D4,97,32.0,n/a,"white, red",red,unknown,n/a,1,[https://swapi.dev/api/films/1/],2,[],[]
8,9,Biggs Darklighter,183,84.0,black,light,brown,24BBY,male,1,[https://swapi.dev/api/films/1/],0,[],[https://swapi.dev/api/starships/12/]
10,11,Anakin Skywalker,188,84.0,blond,fair,blue,41.9BBY,male,1,"[https://swapi.dev/api/films/4/, https://swapi...",0,"[https://swapi.dev/api/vehicles/44/, https://s...","[https://swapi.dev/api/starships/39/, https://..."
41,43,Shmi Skywalker,163,0.0,black,fair,brown,72BBY,female,1,"[https://swapi.dev/api/films/4/, https://swapi...",0,[],[]
60,62,Cliegg Lars,183,0.0,brown,fair,blue,82BBY,male,1,[https://swapi.dev/api/films/5/],0,[],[]


In [67]:
'''They have coherence. We can delete the residents' column from planet_df because the residents are specified in people_df
and it is related to planet_df through planet_id. We also eliminate url, created and edited'''
planets_df.drop(columns=['residents','created','edited','url'],inplace=True)
planets_df.head()

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
0,1,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[https://swapi.dev/api/films/1/]
3,4,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[https://swapi.dev/api/films/2/]
4,5,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,"[https://swapi.dev/api/films/2/, https://swapi..."


In [68]:
#We are going to focus now in the numeric columns (rotation_period, orbital_period, diameter, surface_water and population).
planets_df['rotation_period'].value_counts()

unknown    12
24          6
26          6
27          5
25          5
23          4
20          3
22          3
30          2
12          2
29          2
21          2
36          1
34          1
18          1
87          1
0           1
48          1
33          1
28          1
Name: rotation_period, dtype: int64

In [69]:
#There are all whole numbers. There are 12 unknown and 1 0. We are going to check them.
planets_df[planets_df['rotation_period']=='0']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
27,28,unknown,0,0,0,unknown,unknown,unknown,unknown,unknown,[]


In [70]:
#Planet 28 seems to be an error. We are going to check in people_df if it exists there.
# If not, we are going to eliminate that register.
people_df[people_df['Homeworld_planetId']==28]

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,vehicles,starships
18,20,Yoda,66,17.0,white,green,brown,896BBY,male,28,"[https://swapi.dev/api/films/2/, https://swapi...",6,[],[]
21,23,IG-88,200,140.0,none,metal,red,15BBY,none,28,[https://swapi.dev/api/films/2/],2,[],[]
27,29,Arvel Crynyd,0,0.0,brown,fair,brown,unknown,male,28,[https://swapi.dev/api/films/3/],0,[],[https://swapi.dev/api/starships/28/]
30,32,Qui-Gon Jinn,193,89.0,brown,fair,blue,92BBY,male,28,[https://swapi.dev/api/films/4/],0,[https://swapi.dev/api/vehicles/38/],[]
73,75,R4-P17,96,0.0,none,"silver, red","red, blue",unknown,female,28,"[https://swapi.dev/api/films/5/, https://swapi...",0,[],[]


In [71]:
#Aparently is not an error. It is a register that englobe all the character that have no known homeworld.
#Anyway we can assign 0 to the unknown values of rotation period if we want to in order to transform the dtype.
planets_df[planets_df['rotation_period']=='unknown']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
19,20,Stewjon,unknown,unknown,0,temperate,1 standard,grass,unknown,unknown,[]
37,38,Aleen Minor,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
39,40,Troiken,unknown,unknown,unknown,unknown,unknown,"desert, tundra, rainforests, mountains",unknown,unknown,[]
45,46,Tholoth,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
47,48,Quermia,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
50,51,Mirial,unknown,unknown,unknown,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,unknown,unknown,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]
52,53,Concord Dawn,unknown,unknown,unknown,unknown,unknown,"jungles, forests, deserts",unknown,unknown,[]
53,54,Zolan,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
54,55,Ojom,unknown,unknown,unknown,frigid,unknown,"oceans, glaciers",100,500000000,[]


In [72]:
planets_df['rotation_period'].replace(to_replace='unknown',value=0,inplace=True)
planets_df.tail(10)

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
50,51,Mirial,0,unknown,unknown,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,0,unknown,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]
52,53,Concord Dawn,0,unknown,unknown,unknown,unknown,"jungles, forests, deserts",unknown,unknown,[]
53,54,Zolan,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
54,55,Ojom,0,unknown,unknown,frigid,unknown,"oceans, glaciers",100,500000000,[]
55,56,Skako,27,384,unknown,temperate,1,"urban, vines",unknown,500000000000,[]
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25,5000000000,[]
57,58,Shili,0,unknown,unknown,temperate,1,"cities, savannahs, seas, plains",unknown,unknown,[]
58,59,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",unknown,4000000000,[]
59,60,Umbara,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]


In [73]:
#We asign a number type to the column
planets_df = planets_df.astype({'rotation_period':'int64'})
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Planet_Id        60 non-null     int64 
 1   name             60 non-null     object
 2   rotation_period  60 non-null     int64 
 3   orbital_period   60 non-null     object
 4   diameter         60 non-null     object
 5   climate          60 non-null     object
 6   gravity          60 non-null     object
 7   terrain          60 non-null     object
 8   surface_water    60 non-null     object
 9   population       60 non-null     object
 10  films            60 non-null     object
dtypes: int64(2), object(9)
memory usage: 5.3+ KB


In [74]:
#We do the same analysis with the other numerical columns
planets_df['orbital_period'].value_counts()

unknown    12
305         2
368         2
378         2
412         2
413         2
263         1
680         1
334         1
0           1
318         1
371         1
326         1
398         1
206         1
386         1
201         1
409         1
491         1
391         1
1770        1
481         1
383         1
184         1
304         1
364         1
256         1
4818        1
549         1
341         1
5110        1
402         1
312         1
463         1
351         1
329         1
381         1
590         1
167         1
231         1
278         1
392         1
360         1
384         1
Name: orbital_period, dtype: int64

In [75]:
planets_df[planets_df['orbital_period']=='0']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
27,28,unknown,0,0,0,unknown,unknown,unknown,unknown,unknown,[]


In [76]:
planets_df[planets_df['orbital_period']=='unknown']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
19,20,Stewjon,0,unknown,0,temperate,1 standard,grass,unknown,unknown,[]
37,38,Aleen Minor,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
39,40,Troiken,0,unknown,unknown,unknown,unknown,"desert, tundra, rainforests, mountains",unknown,unknown,[]
45,46,Tholoth,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
47,48,Quermia,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
50,51,Mirial,0,unknown,unknown,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,0,unknown,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]
52,53,Concord Dawn,0,unknown,unknown,unknown,unknown,"jungles, forests, deserts",unknown,unknown,[]
53,54,Zolan,0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,[]
54,55,Ojom,0,unknown,unknown,frigid,unknown,"oceans, glaciers",100,500000000,[]


In [77]:
planets_df['orbital_period'].replace(to_replace='unknown',value=0,inplace=True)
planets_df.tail(10)

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
50,51,Mirial,0,0,unknown,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,0,0,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]
52,53,Concord Dawn,0,0,unknown,unknown,unknown,"jungles, forests, deserts",unknown,unknown,[]
53,54,Zolan,0,0,unknown,unknown,unknown,unknown,unknown,unknown,[]
54,55,Ojom,0,0,unknown,frigid,unknown,"oceans, glaciers",100,500000000,[]
55,56,Skako,27,384,unknown,temperate,1,"urban, vines",unknown,500000000000,[]
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25,5000000000,[]
57,58,Shili,0,0,unknown,temperate,1,"cities, savannahs, seas, plains",unknown,unknown,[]
58,59,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",unknown,4000000000,[]
59,60,Umbara,0,0,unknown,unknown,unknown,unknown,unknown,unknown,[]


In [78]:
planets_df = planets_df.astype({'orbital_period':'int64'})
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Planet_Id        60 non-null     int64 
 1   name             60 non-null     object
 2   rotation_period  60 non-null     int64 
 3   orbital_period   60 non-null     int64 
 4   diameter         60 non-null     object
 5   climate          60 non-null     object
 6   gravity          60 non-null     object
 7   terrain          60 non-null     object
 8   surface_water    60 non-null     object
 9   population       60 non-null     object
 10  films            60 non-null     object
dtypes: int64(3), object(8)
memory usage: 5.3+ KB


In [79]:
planets_df['diameter'].value_counts()

unknown    16
0           6
10465       1
18880       1
6400        1
14050       1
11030       1
13500       1
12780       1
7900        1
10480       1
12150       1
10600       1
14900       1
12190       1
10120       1
15600       1
13400       1
13800       1
9830        1
7549        1
12500       1
19720       1
10200       1
7200        1
8900        1
118000      1
4900        1
12120       1
12240       1
11370       1
11000       1
12900       1
4200        1
12765       1
10088       1
9100        1
14920       1
13490       1
13850       1
Name: diameter, dtype: int64

In [80]:
planets_df[planets_df['diameter']=='0']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
14,15,Polis Massa,24,590,0,artificial temperate,0.56 standard,airless asteroid,0,1000000,[https://swapi.dev/api/films/6/]
17,18,Cato Neimoidia,25,278,0,"temperate, moist",1 standard,"mountains, fields, forests, rock arches",unknown,10000000,[https://swapi.dev/api/films/6/]
19,20,Stewjon,0,0,0,temperate,1 standard,grass,unknown,unknown,[]
27,28,unknown,0,0,0,unknown,unknown,unknown,unknown,unknown,[]
28,29,Trandosha,25,371,0,arid,0.62 standard,"mountains, seas, grasslands, deserts",unknown,42000000,[]
29,30,Socorro,20,326,0,arid,1 standard,"deserts, mountains",unknown,300000000,[]


In [81]:
planets_df[planets_df['diameter']=='unknown']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
37,38,Aleen Minor,0,0,unknown,unknown,unknown,unknown,unknown,unknown,[]
39,40,Troiken,0,0,unknown,unknown,unknown,"desert, tundra, rainforests, mountains",unknown,unknown,[]
42,43,Cerea,27,386,unknown,temperate,1,verdant,20,450000000,[]
44,45,Iridonia,29,413,unknown,unknown,unknown,"rocky canyons, acid pools",unknown,unknown,[]
45,46,Tholoth,0,0,unknown,unknown,unknown,unknown,unknown,unknown,[]
46,47,Iktotch,22,481,unknown,"arid, rocky, windy",1,rocky,unknown,unknown,[]
47,48,Quermia,0,0,unknown,unknown,unknown,unknown,unknown,unknown,[]
49,50,Champala,27,318,unknown,temperate,1,"oceans, rainforests, plateaus",unknown,3500000000,[]
50,51,Mirial,0,0,unknown,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,0,0,unknown,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]


In [82]:
planets_df['diameter'].replace(to_replace='unknown',value=0,inplace=True)
planets_df.tail(10)

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
50,51,Mirial,0,0,0,unknown,unknown,deserts,unknown,unknown,[]
51,52,Serenno,0,0,0,unknown,unknown,"rainforests, rivers, mountains",unknown,unknown,[]
52,53,Concord Dawn,0,0,0,unknown,unknown,"jungles, forests, deserts",unknown,unknown,[]
53,54,Zolan,0,0,0,unknown,unknown,unknown,unknown,unknown,[]
54,55,Ojom,0,0,0,frigid,unknown,"oceans, glaciers",100,500000000,[]
55,56,Skako,27,384,0,temperate,1,"urban, vines",unknown,500000000000,[]
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25,5000000000,[]
57,58,Shili,0,0,0,temperate,1,"cities, savannahs, seas, plains",unknown,unknown,[]
58,59,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",unknown,4000000000,[]
59,60,Umbara,0,0,0,unknown,unknown,unknown,unknown,unknown,[]


In [83]:
planets_df = planets_df.astype({'diameter':'int64'})
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Planet_Id        60 non-null     int64 
 1   name             60 non-null     object
 2   rotation_period  60 non-null     int64 
 3   orbital_period   60 non-null     int64 
 4   diameter         60 non-null     int64 
 5   climate          60 non-null     object
 6   gravity          60 non-null     object
 7   terrain          60 non-null     object
 8   surface_water    60 non-null     object
 9   population       60 non-null     object
 10  films            60 non-null     object
dtypes: int64(4), object(7)
memory usage: 5.3+ KB


In [84]:
planets_df['surface_water'].value_counts()

unknown    34
100         4
8           3
0           3
5           3
40          2
60          2
1           1
12          1
0.9         1
70          1
98          1
10          1
20          1
80          1
25          1
Name: surface_water, dtype: int64

In [85]:
planets_df[planets_df['surface_water']=='0']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
5,6,Bespin,12,5110,118000,temperate,"1.5 (surface), 1 standard (Cloud City)",gas giant,0,6000000,[https://swapi.dev/api/films/2/]
12,13,Mustafar,36,412,4200,hot,1 standard,"volcanoes, lava rivers, mountains, caves",0,20000,[https://swapi.dev/api/films/6/]
14,15,Polis Massa,24,590,0,artificial temperate,0.56 standard,airless asteroid,0,1000000,[https://swapi.dev/api/films/6/]


In [86]:
#On this case I can't assign to the unknown value the value of 0 because there are planets that are known to have 0 of water surface.
#We are going to assign the unknown values the value -1.
planets_df[planets_df['surface_water']=='unknown']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
8,9,Coruscant,24,368,12240,temperate,1 standard,"cityscape, mountains",unknown,1000000000000,"[https://swapi.dev/api/films/3/, https://swapi..."
15,16,Mygeeto,12,167,10088,frigid,1 standard,"glaciers, mountains, ice canyons",unknown,19000000,[https://swapi.dev/api/films/6/]
16,17,Felucia,34,231,9100,"hot, humid",0.75 standard,fungus forests,unknown,8500000,[https://swapi.dev/api/films/6/]
17,18,Cato Neimoidia,25,278,0,"temperate, moist",1 standard,"mountains, fields, forests, rock arches",unknown,10000000,[https://swapi.dev/api/films/6/]
18,19,Saleucami,26,392,14920,hot,unknown,"caves, desert, mountains, volcanoes",unknown,1400000000,[https://swapi.dev/api/films/6/]
19,20,Stewjon,0,0,0,temperate,1 standard,grass,unknown,unknown,[]
20,21,Eriadu,24,360,13490,polluted,1 standard,cityscape,unknown,22000000000,[]
23,24,Nal Hutta,87,413,12150,temperate,1 standard,"urban, oceans, swamps, bogs",unknown,7000000000,[]
24,25,Dantooine,25,378,9830,temperate,1 standard,"oceans, savannas, mountains, grasslands",unknown,1000,[]
27,28,unknown,0,0,0,unknown,unknown,unknown,unknown,unknown,[]


In [87]:
#We are going to assign it the value of -1 but we must report that more than half of the registers have unknowns values
#To eliminate the field is a possibility
planets_df['surface_water'].replace(to_replace='unknown',value=-1,inplace=True)
planets_df.tail(10)

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
50,51,Mirial,0,0,0,unknown,unknown,deserts,-1,unknown,[]
51,52,Serenno,0,0,0,unknown,unknown,"rainforests, rivers, mountains",-1,unknown,[]
52,53,Concord Dawn,0,0,0,unknown,unknown,"jungles, forests, deserts",-1,unknown,[]
53,54,Zolan,0,0,0,unknown,unknown,unknown,-1,unknown,[]
54,55,Ojom,0,0,0,frigid,unknown,"oceans, glaciers",100,500000000,[]
55,56,Skako,27,384,0,temperate,1,"urban, vines",-1,500000000000,[]
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25,5000000000,[]
57,58,Shili,0,0,0,temperate,1,"cities, savannahs, seas, plains",-1,unknown,[]
58,59,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",-1,4000000000,[]
59,60,Umbara,0,0,0,unknown,unknown,unknown,-1,unknown,[]


In [88]:
planets_df = planets_df.astype({'surface_water':'float64'})
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Planet_Id        60 non-null     int64  
 1   name             60 non-null     object 
 2   rotation_period  60 non-null     int64  
 3   orbital_period   60 non-null     int64  
 4   diameter         60 non-null     int64  
 5   climate          60 non-null     object 
 6   gravity          60 non-null     object 
 7   terrain          60 non-null     object 
 8   surface_water    60 non-null     float64
 9   population       60 non-null     object 
 10  films            60 non-null     object 
dtypes: float64(1), int64(4), object(6)
memory usage: 5.3+ KB


In [89]:
planets_df['population'].value_counts()

unknown          17
2000000000        2
1000              2
4000000000        2
500000000         2
5200              1
42000000          1
300000000         1
27000000000       1
1200000000        1
18500000000       1
11000000          1
200000            1
1500000000        1
62000000          1
0                 1
705300            1
450000000         1
3500000000        1
500000000000      1
421000000         1
1300000000        1
7000000000        1
3000000000        1
6000000           1
30000000          1
4500000000        1
1000000000000     1
1000000000        1
100000000000      1
95000000          1
20000             1
45000000          1
1000000           1
19000000          1
8500000           1
10000000          1
1400000000        1
22000000000       1
5000000000        1
Name: population, dtype: int64

In [90]:
planets_df[planets_df['population']=='0']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
40,41,Tund,48,1770,12190,unknown,unknown,"barren, ash",-1.0,0,[]


In [91]:
#Same analysis as with surface_water. Maybe this planet is known to be inhabited. We assign the unknown -1.
planets_df[planets_df['population']=='unknown']

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
3,4,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100.0,unknown,[https://swapi.dev/api/films/2/]
4,5,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8.0,unknown,"[https://swapi.dev/api/films/2/, https://swapi..."
19,20,Stewjon,0,0,0,temperate,1 standard,grass,-1.0,unknown,[]
27,28,unknown,0,0,0,unknown,unknown,unknown,-1.0,unknown,[]
37,38,Aleen Minor,0,0,0,unknown,unknown,unknown,-1.0,unknown,[]
39,40,Troiken,0,0,0,unknown,unknown,"desert, tundra, rainforests, mountains",-1.0,unknown,[]
44,45,Iridonia,29,413,0,unknown,unknown,"rocky canyons, acid pools",-1.0,unknown,[]
45,46,Tholoth,0,0,0,unknown,unknown,unknown,-1.0,unknown,[]
46,47,Iktotch,22,481,0,"arid, rocky, windy",1,rocky,-1.0,unknown,[]
47,48,Quermia,0,0,0,unknown,unknown,unknown,-1.0,unknown,[]


In [92]:
planets_df['population'].replace(to_replace='unknown',value=-1,inplace=True)
planets_df.tail(10)

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
50,51,Mirial,0,0,0,unknown,unknown,deserts,-1.0,-1,[]
51,52,Serenno,0,0,0,unknown,unknown,"rainforests, rivers, mountains",-1.0,-1,[]
52,53,Concord Dawn,0,0,0,unknown,unknown,"jungles, forests, deserts",-1.0,-1,[]
53,54,Zolan,0,0,0,unknown,unknown,unknown,-1.0,-1,[]
54,55,Ojom,0,0,0,frigid,unknown,"oceans, glaciers",100.0,500000000,[]
55,56,Skako,27,384,0,temperate,1,"urban, vines",-1.0,500000000000,[]
56,57,Muunilinst,28,412,13800,temperate,1,"plains, forests, hills, mountains",25.0,5000000000,[]
57,58,Shili,0,0,0,temperate,1,"cities, savannahs, seas, plains",-1.0,-1,[]
58,59,Kalee,23,378,13850,"arid, temperate, tropical",1,"rainforests, cliffs, canyons, seas",-1.0,4000000000,[]
59,60,Umbara,0,0,0,unknown,unknown,unknown,-1.0,-1,[]


In [93]:
planets_df = planets_df.astype({'population':'int64'})
planets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Planet_Id        60 non-null     int64  
 1   name             60 non-null     object 
 2   rotation_period  60 non-null     int64  
 3   orbital_period   60 non-null     int64  
 4   diameter         60 non-null     int64  
 5   climate          60 non-null     object 
 6   gravity          60 non-null     object 
 7   terrain          60 non-null     object 
 8   surface_water    60 non-null     float64
 9   population       60 non-null     int64  
 10  films            60 non-null     object 
dtypes: float64(1), int64(5), object(5)
memory usage: 5.3+ KB


In [94]:
planets_df.head()

,Planet_Id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,films
0,1,Tatooine,23,304,10465,arid,1 standard,desert,1.0,200000,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40.0,2000000000,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8.0,1000,[https://swapi.dev/api/films/1/]
3,4,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100.0,-1,[https://swapi.dev/api/films/2/]
4,5,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8.0,-1,"[https://swapi.dev/api/films/2/, https://swapi..."


In [95]:
'''We finish with planets. Films we are going to skip it for the moment because it has a many-to-many relationship with 
the other tables. We continue with species_df'''
species_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              37 non-null     object
 1   classification    37 non-null     object
 2   designation       37 non-null     object
 3   average_height    37 non-null     object
 4   skin_colors       37 non-null     object
 5   hair_colors       37 non-null     object
 6   eye_colors        37 non-null     object
 7   average_lifespan  37 non-null     object
 8   homeworld         36 non-null     object
 9   language          37 non-null     object
 10  people            37 non-null     object
 11  films             37 non-null     object
 12  created           37 non-null     object
 13  edited            37 non-null     object
 14  url               37 non-null     object
dtypes: object(15)
memory usage: 4.5+ KB


In [96]:
species_df.head()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/


In [97]:
#We begin as always defining an Id from the url.
species_df['url'][0].index('1')

30

In [98]:
sp_ids = []
for i in range(species_df.shape[0]):
    sp_ids.append(int(species_df['url'][i][30:-1]))
sp_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [99]:
species_df.insert(loc=0,column='Species_Id',value=sp_ids)
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/
1,2,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/


In [100]:
#We analize the homeworld.
species_df['homeworld']

0      https://swapi.dev/api/planets/9/
1                                  None
2     https://swapi.dev/api/planets/14/
3     https://swapi.dev/api/planets/23/
4     https://swapi.dev/api/planets/24/
5     https://swapi.dev/api/planets/28/
6     https://swapi.dev/api/planets/29/
7     https://swapi.dev/api/planets/31/
8      https://swapi.dev/api/planets/7/
9     https://swapi.dev/api/planets/33/
10    https://swapi.dev/api/planets/18/
11     https://swapi.dev/api/planets/8/
12    https://swapi.dev/api/planets/34/
13    https://swapi.dev/api/planets/35/
14    https://swapi.dev/api/planets/37/
15    https://swapi.dev/api/planets/38/
16    https://swapi.dev/api/planets/39/
17    https://swapi.dev/api/planets/40/
18    https://swapi.dev/api/planets/41/
19    https://swapi.dev/api/planets/43/
20    https://swapi.dev/api/planets/44/
21    https://swapi.dev/api/planets/45/
22    https://swapi.dev/api/planets/46/
23    https://swapi.dev/api/planets/47/
24    https://swapi.dev/api/planets/48/


In [101]:
#Everyone has one planet except the second register (Droid). So we identify it with the planet id but we skip the second row.
species_df['homeworld'][0].index('9')

30

In [102]:
#To the Droids we are going to assign 0 as homeworld Id
pl_ids = []
for i in range(species_df.shape[0]):
    if i == 1:
        pl_ids.append(0)
    else:
        pl_ids.append(int(species_df['homeworld'][i][30:-1]))
pl_ids

[9,
 0,
 14,
 23,
 24,
 28,
 29,
 31,
 7,
 33,
 18,
 8,
 34,
 35,
 37,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 11,
 51,
 54,
 55,
 10,
 56,
 57,
 58,
 59,
 12]

In [103]:
species_df.insert(loc=9,column='Planet_Id',value=pl_ids)
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,Planet_Id,homeworld,language,people,films,created,edited,url
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/
1,2,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,0,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,23,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/


In [104]:
'''We are going to discard all the information that is redundant like the fields homeworld, url and people (we allready
check that people_df contains species_id) and we are going to also drop (as usual) created and edited'''
species_df.drop(columns=['homeworld','people','created','edited','url'],inplace=True)
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,Planet_Id,language,films
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,Galactic Basic,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,0,n/a,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,Shyriiwook,"[https://swapi.dev/api/films/1/, https://swapi..."
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,23,Galatic Basic,[https://swapi.dev/api/films/1/]
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,Huttese,"[https://swapi.dev/api/films/1/, https://swapi..."


In [105]:
#We analyze the numeric columns (average_height and lifespan).
species_df['average_height'].value_counts()

180        10
200         4
190         4
100         3
170         2
unknown     2
178         2
300         1
66          1
160         1
n/a         1
210         1
120         1
80          1
125         1
240         1
220         1
Name: average_height, dtype: int64

In [106]:
species_df['average_lifespan'].value_counts()

unknown       20
70             3
80             2
120            1
indefinite     1
400            1
1000           1
900            1
91             1
79             1
86             1
75             1
100            1
94             1
700            1
Name: average_lifespan, dtype: int64

In [107]:
#Naturally has, the droids, no average_height(n/a) and life_span(indefinite). We are going to replace these values with -1.
species_df['average_height'].replace(to_replace='n/a',value=-1,inplace=True)
species_df['average_lifespan'].replace(to_replace='indefinite',value=-1,inplace=True)
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,Planet_Id,language,films
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,Galactic Basic,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Droid,artificial,sentient,-1,n/a,n/a,n/a,-1,0,n/a,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,Shyriiwook,"[https://swapi.dev/api/films/1/, https://swapi..."
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,23,Galatic Basic,[https://swapi.dev/api/films/1/]
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,Huttese,"[https://swapi.dev/api/films/1/, https://swapi..."


In [108]:
#The unknown values we are replacing with 0. We notice that more than the half of the values of average_lifespan are unknown.
species_df['average_height'].replace(to_replace='unknown',value=0,inplace=True)
species_df['average_lifespan'].replace(to_replace='unknown',value=0,inplace=True)
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,Planet_Id,language,films
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,Galactic Basic,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Droid,artificial,sentient,-1,n/a,n/a,n/a,-1,0,n/a,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,Shyriiwook,"[https://swapi.dev/api/films/1/, https://swapi..."
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,0,23,Galatic Basic,[https://swapi.dev/api/films/1/]
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,Huttese,"[https://swapi.dev/api/films/1/, https://swapi..."


In [109]:
species_df = species_df.astype({'average_height':'int64','average_lifespan':'int64'})
species_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Species_Id        37 non-null     int64 
 1   name              37 non-null     object
 2   classification    37 non-null     object
 3   designation       37 non-null     object
 4   average_height    37 non-null     int64 
 5   skin_colors       37 non-null     object
 6   hair_colors       37 non-null     object
 7   eye_colors        37 non-null     object
 8   average_lifespan  37 non-null     int64 
 9   Planet_Id         37 non-null     int64 
 10  language          37 non-null     object
 11  films             37 non-null     object
dtypes: int64(4), object(8)
memory usage: 3.6+ KB


In [110]:
species_df.head()

,Species_Id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,Planet_Id,language,films
0,1,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,Galactic Basic,"[https://swapi.dev/api/films/1/, https://swapi..."
1,2,Droid,artificial,sentient,-1,n/a,n/a,n/a,-1,0,n/a,"[https://swapi.dev/api/films/1/, https://swapi..."
2,3,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,Shyriiwook,"[https://swapi.dev/api/films/1/, https://swapi..."
3,4,Rodian,sentient,reptilian,170,"green, blue",n/a,black,0,23,Galatic Basic,[https://swapi.dev/api/films/1/]
4,5,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,Huttese,"[https://swapi.dev/api/films/1/, https://swapi..."


In [111]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    39 non-null     object
 1   model                   39 non-null     object
 2   manufacturer            39 non-null     object
 3   cost_in_credits         39 non-null     object
 4   length                  39 non-null     object
 5   max_atmosphering_speed  39 non-null     object
 6   crew                    39 non-null     object
 7   passengers              39 non-null     object
 8   cargo_capacity          39 non-null     object
 9   consumables             39 non-null     object
 10  vehicle_class           39 non-null     object
 11  pilots                  39 non-null     object
 12  films                   39 non-null     object
 13  created                 39 non-null     object
 14  edited                  39 non-null     object
 15  url     

In [112]:
vehicles_df.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,https://swapi.dev/api/vehicles/4/
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:01:52.434000Z,2014-12-20T21:30:21.665000Z,https://swapi.dev/api/vehicles/6/
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:13:52.586000Z,2014-12-20T21:30:21.668000Z,https://swapi.dev/api/vehicles/7/
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:33:52.860000Z,2014-12-20T21:30:21.670000Z,https://swapi.dev/api/vehicles/8/
4,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"[https://swapi.dev/api/people/1/, https://swap...",[https://swapi.dev/api/films/2/],2014-12-15T12:22:12Z,2014-12-20T21:30:21.672000Z,https://swapi.dev/api/vehicles/14/


In [113]:
#We assing an Id from the url
vehicles_df['url'][0].index('4')

31

In [114]:
ve_ids = []
for i in range(vehicles_df.shape[0]):
    ve_ids.append(int(vehicles_df['url'][i][31:-1]))
ve_ids

[4,
 6,
 7,
 8,
 14,
 16,
 18,
 19,
 20,
 24,
 25,
 26,
 30,
 33,
 34,
 35,
 36,
 37,
 38,
 42,
 44,
 45,
 46,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 60,
 62,
 67,
 69,
 70,
 71,
 72,
 73,
 76]

In [115]:
#For a strange reason the ids are not continuous. We insert them.
vehicles_df.insert(loc=0,column='Veh_Id',value=ve_ids)
vehicles_df.head()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,4,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,https://swapi.dev/api/vehicles/4/
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:01:52.434000Z,2014-12-20T21:30:21.665000Z,https://swapi.dev/api/vehicles/6/
2,7,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[],[https://swapi.dev/api/films/1/],2014-12-10T16:13:52.586000Z,2014-12-20T21:30:21.668000Z,https://swapi.dev/api/vehicles/7/
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:33:52.860000Z,2014-12-20T21:30:21.670000Z,https://swapi.dev/api/vehicles/8/
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"[https://swapi.dev/api/people/1/, https://swap...",[https://swapi.dev/api/films/2/],2014-12-15T12:22:12Z,2014-12-20T21:30:21.672000Z,https://swapi.dev/api/vehicles/14/


In [116]:
#We look into the field pilots
vehicles_df['pilots']

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4     [https://swapi.dev/api/people/1/, https://swap...
5                                                    []
6                                                    []
7                    [https://swapi.dev/api/people/13/]
8                                                    []
9                                                    []
10                                                   []
11                                                   []
12    [https://swapi.dev/api/people/1/, https://swap...
13                                                   []
14                                                   []
15                                                   []
16                                                   []
17                                              

In [117]:
#There are 3 registers that have more than 1 pilot. Let's check them.
print(vehicles_df['pilots'][4])
print(vehicles_df['pilots'][12])
print(vehicles_df['pilots'][18])

['https://swapi.dev/api/people/1/', 'https://swapi.dev/api/people/18/']
['https://swapi.dev/api/people/1/', 'https://swapi.dev/api/people/5/']
['https://swapi.dev/api/people/10/', 'https://swapi.dev/api/people/32/']


In [118]:
#We are checking for the column vehicles in people_df but we are only checking with this people ids
people_df[people_df['People_Id'].isin([1,5,10,18,32])]['vehicles']

0     [https://swapi.dev/api/vehicles/14/, https://s...
4                  [https://swapi.dev/api/vehicles/30/]
9                  [https://swapi.dev/api/vehicles/38/]
16                 [https://swapi.dev/api/vehicles/14/]
30                 [https://swapi.dev/api/vehicles/38/]
Name: vehicles, dtype: object

In [119]:
#We check the values under the column vehicles
people_df['vehicles'].value_counts()

[]                                                                          71
[https://swapi.dev/api/vehicles/38/]                                         2
[https://swapi.dev/api/vehicles/14/, https://swapi.dev/api/vehicles/30/]     1
[https://swapi.dev/api/vehicles/30/]                                         1
[https://swapi.dev/api/vehicles/44/, https://swapi.dev/api/vehicles/46/]     1
[https://swapi.dev/api/vehicles/19/]                                         1
[https://swapi.dev/api/vehicles/14/]                                         1
[https://swapi.dev/api/vehicles/42/]                                         1
[https://swapi.dev/api/vehicles/55/]                                         1
[https://swapi.dev/api/vehicles/45/]                                         1
[https://swapi.dev/api/vehicles/60/]                                         1
Name: vehicles, dtype: int64

There 2 characters that drive 2 vehicles and 3 vehicles that have 2 pilots. The relationship between this 2 is many to many. We would need another table to fix this but there are also 2 simple ways of solving this. We could add a new column to people_df (2nd vehicle) or add a new column to vehicles_df (2nd pilot). We are going to do the first one but only in order to show this possible solution.

In [120]:
#We make a list with the numbers of the first vehicles. We check first the table.
people_df.head()

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,[],[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",2,[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,[],[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,[https://swapi.dev/api/vehicles/30/],[]


In [121]:
#We find where it begin the number
people_df['vehicles'][0][0].index('1')

31

In [122]:
#We make a list with the first vehicles. The ones that don't have any we fill it with 0.
first_veh_ids = []
for i in range(people_df.shape[0]):
    if len(people_df['vehicles'][i]) == 0:
        first_veh_ids.append(0)
    else:
        first_veh_ids.append(int(people_df['vehicles'][i][0][31:-1]))
first_veh_ids

[14,
 0,
 0,
 0,
 30,
 0,
 0,
 0,
 0,
 38,
 44,
 0,
 19,
 0,
 0,
 0,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 42,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 55,
 0,
 0,
 45,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 60,
 0,
 0,
 0,
 0]

In [123]:
sec_veh_ids = []
for i in range(people_df.shape[0]):
    if len(people_df['vehicles'][i]) < 2:
        sec_veh_ids.append(0)
    else:
        sec_veh_ids.append(int(people_df['vehicles'][i][1][31:-1]))
sec_veh_ids

[30,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 46,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [124]:
#We insert them.
people_df.insert(loc=12,column='First_vehicle_Id',value=first_veh_ids)

In [125]:
people_df.insert(loc=13,column='Second_vehicle_Id',value=sec_veh_ids)

In [126]:
people_df.head()

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,First_vehicle_Id,Second_vehicle_Id,vehicles,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,14,30,"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,0,0,[],[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",2,0,0,[],[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,0,0,[],[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,30,0,[https://swapi.dev/api/vehicles/30/],[]


In [127]:
#Now we can drop the vehicle column in people_df and the pilot column in vehicle_df
people_df.drop(columns='vehicles',inplace=True)
people_df.head()

,People_Id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,Homeworld_planetId,films,Species_Id,First_vehicle_Id,Second_vehicle_Id,starships
0,1,Luke Skywalker,172,77.0,blond,fair,blue,19BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,14,30,"[https://swapi.dev/api/starships/12/, https://..."
1,2,C-3PO,167,75.0,n/a,gold,yellow,112BBY,n/a,1,"[https://swapi.dev/api/films/1/, https://swapi...",2,0,0,[]
2,3,R2-D2,96,32.0,n/a,"white, blue",red,33BBY,n/a,8,"[https://swapi.dev/api/films/1/, https://swapi...",2,0,0,[]
3,4,Darth Vader,202,136.0,none,white,yellow,41.9BBY,male,1,"[https://swapi.dev/api/films/1/, https://swapi...",0,0,0,[https://swapi.dev/api/starships/13/]
4,5,Leia Organa,150,49.0,brown,light,brown,19BBY,female,2,"[https://swapi.dev/api/films/1/, https://swapi...",0,30,0,[]


In [128]:
#We now drop the redundant columns in vehicles_df.
vehicles_df.drop(columns=['pilots','created','edited','url'],inplace=True)
vehicles_df.head()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
0,4,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,"[https://swapi.dev/api/films/1/, https://swapi..."
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[https://swapi.dev/api/films/1/]
2,7,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[https://swapi.dev/api/films/1/]
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,"[https://swapi.dev/api/films/1/, https://swapi..."
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,[https://swapi.dev/api/films/2/]


In [129]:
#We focus now in the numeric columns.
vehicles_df['cost_in_credits'].value_counts()

unknown    18
8000        3
150000      1
5750        1
60000       1
49000       1
350000      1
14750       1
12125       1
20000       1
15000       1
4000        1
14500       1
200000      1
2500        1
138000      1
285000      1
75000       1
10550       1
40000       1
Name: cost_in_credits, dtype: int64

In [130]:
vehicles_df[vehicles_df['cost_in_credits']=='unknown']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,"[https://swapi.dev/api/films/1/, https://swapi..."
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,[https://swapi.dev/api/films/2/]
5,16,TIE bomber,TIE/sa bomber,Sienar Fleet Systems,unknown,7.8,850,1,0,none,2 days,space/planetary bomber,"[https://swapi.dev/api/films/2/, https://swapi..."
6,18,AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,20,60,5,40,1000,unknown,assault walker,"[https://swapi.dev/api/films/2/, https://swapi..."
7,19,AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,2,90,2,0,200,none,walker,"[https://swapi.dev/api/films/2/, https://swapi..."
11,26,TIE/IN interceptor,Twin Ion Engine Interceptor,Sienar Fleet Systems,unknown,9.6,1250,1,0,75,2 days,starfighter,[https://swapi.dev/api/films/3/]
13,33,Vulture Droid,Vulture-class droid starfighter,"Haor Chall Engineering, Baktoid Armor Workshop",unknown,3.5,1200,0,0,0,none,starfighter,"[https://swapi.dev/api/films/4/, https://swapi..."
15,35,Armored Assault Tank,Armoured Assault Tank,Baktoid Armor Workshop,unknown,9.75,55,4,6,unknown,unknown,repulsorcraft,[https://swapi.dev/api/films/4/]
18,38,Tribubble bongo,Tribubble bongo,Otoh Gunga Bongameken Cooperative,unknown,15,85,1,2,1600,unknown,submarine,[https://swapi.dev/api/films/4/]
21,45,Koro-2 Exodrive airspeeder,Koro-2 Exodrive airspeeder,Desler Gizh Outworld Mobility Corporation,unknown,6.6,800,1,1,80,unknown,airspeeder,[https://swapi.dev/api/films/5/]


In [131]:
#We give it the value of -1.
vehicles_df['cost_in_credits'].replace(to_replace='unknown',value=-1,inplace=True)
vehicles_df.head()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
0,4,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,"[https://swapi.dev/api/films/1/, https://swapi..."
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[https://swapi.dev/api/films/1/]
2,7,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[https://swapi.dev/api/films/1/]
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,-1,6.4,1200,1,0,65,2 days,starfighter,"[https://swapi.dev/api/films/1/, https://swapi..."
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,-1,4.5,650,2,0,10,none,airspeeder,[https://swapi.dev/api/films/2/]


In [132]:
vehicles_df['length'].value_counts()

2          3
20         2
7          2
3.5        2
36.8       1
9.8        1
6.23       1
17.4       1
28.82      1
13.2       1
140        1
5.4        1
unknown    1
3.68       1
15.1       1
49.4       1
10.96      1
12.3       1
6.6        1
15         1
1.5        1
10.4       1
210        1
9.75       1
31         1
3          1
9.6        1
9.5        1
30         1
7.8        1
4.5        1
6.4        1
3.4        1
3.2        1
Name: length, dtype: int64

In [133]:
vehicles_df[vehicles_df['length']=='unknown']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,unknown,unknown,2,unknown,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [134]:
#We give it 0 to the unknown value.
vehicles_df['length'].replace(to_replace='unknown',value=0,inplace=True)
vehicles_df[vehicles_df['name']=='Emergency Firespeeder']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,unknown,2,unknown,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [135]:
vehicles_df['max_atmosphering_speed'].value_counts()

1200       3
620        2
35         2
250        2
60         2
90         2
100        2
634        1
880        1
20000      1
330        1
30         1
unknown    1
720        1
420        1
310        1
160        1
1180       1
85         1
800        1
350        1
180        1
587        1
400        1
55         1
360        1
1250       1
1500       1
850        1
650        1
820        1
Name: max_atmosphering_speed, dtype: int64

In [136]:
vehicles_df[vehicles_df['max_atmosphering_speed']=='unknown']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,unknown,2,unknown,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [137]:
#We give it 0 to the unknown value.
vehicles_df['max_atmosphering_speed'].replace(to_replace='unknown',value=0,inplace=True)
vehicles_df[vehicles_df['name']=='Emergency Firespeeder']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,0,2,unknown,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [138]:
vehicles_df['crew'].value_counts()

1      18
2       7
0       3
5       2
4       2
6       2
46      1
26      1
140     1
25      1
20      1
Name: crew, dtype: int64

In [139]:
vehicles_df[vehicles_df['crew']=='0']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
13,33,Vulture Droid,Vulture-class droid starfighter,"Haor Chall Engineering, Baktoid Armor Workshop",-1,3.5,1200,0,0,0,none,starfighter,"[https://swapi.dev/api/films/4/, https://swapi..."
36,72,Corporate Alliance tank droid,NR-N99 Persuader-class droid enforcer,Techno Union,49000,10.96,100,0,4,none,none,droid tank,[https://swapi.dev/api/films/6/]
37,73,Droid gunship,HMP droid gunship,"Baktoid Fleet Ordnance, Haor Chall Engineering",60000,12.3,820,0,0,0,none,airspeeder,[https://swapi.dev/api/films/6/]


In [140]:
vehicles_df['passengers'].value_counts()

0          16
1           7
30          3
6           2
2           2
40          1
500         1
16          1
112         1
284         1
36          1
unknown     1
300         1
4           1
Name: passengers, dtype: int64

In [141]:
vehicles_df[vehicles_df['passengers']=='unknown']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,0,2,unknown,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [142]:
#We give it the unknown the value of -1.
vehicles_df['passengers'].replace(to_replace='unknown',value=-1,inplace=True)
vehicles_df[vehicles_df['name']=='Emergency Firespeeder']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,0,2,-1,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [143]:
vehicles_df['cargo_capacity'].value_counts()

unknown    5
0          3
10         3
none       3
20         2
1000       2
200        2
50         2
1600       1
500        1
10000      1
40000      1
170        1
80         1
2          1
50000      1
1800000    1
12000      1
4          1
75         1
135000     1
2000000    1
65         1
5          1
30000      1
Name: cargo_capacity, dtype: int64

In [144]:
vehicles_df[vehicles_df['cargo_capacity']=='unknown']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
15,35,Armored Assault Tank,Armoured Assault Tank,Baktoid Armor Workshop,-1,9.75,55,4,6,unknown,unknown,repulsorcraft,[https://swapi.dev/api/films/4/]
22,46,XJ-6 airspeeder,XJ-6 airspeeder,Narglatch AirTech prefabricated kit,-1,6.23,720,1,1,unknown,unknown,airspeeder,[https://swapi.dev/api/films/5/]
27,55,Flitknot speeder,Flitknot speeder,Huppla Pasa Tisc Shipwrights Collective,8000,2,634,1,0,unknown,unknown,speeder,[https://swapi.dev/api/films/5/]
29,57,Geonosian starfighter,Nantex-class territorial defense,Huppla Pasa Tisc Shipwrights Collective,-1,9.8,20000,1,0,unknown,unknown,starfighter,[https://swapi.dev/api/films/5/]
31,62,Emergency Firespeeder,Fire suppression speeder,unknown,-1,0,0,2,-1,unknown,unknown,fire suppression ship,[https://swapi.dev/api/films/6/]


In [145]:
#We give it the unknown the value of -1.
vehicles_df['cargo_capacity'].replace(to_replace='unknown',value=-1,inplace=True)
vehicles_df.tail()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
34,70,Raddaugh Gnasp fluttercraft,Raddaugh Gnasp fluttercraft,Appazanna Engineering Works,14750,7,310,2,0,20,none,air speeder,[https://swapi.dev/api/films/6/]
35,71,Clone turbo tank,HAVw A6 Juggernaut,Kuat Drive Yards,350000,49.4,160,20,300,30000,20 days,wheeled walker,[https://swapi.dev/api/films/6/]
36,72,Corporate Alliance tank droid,NR-N99 Persuader-class droid enforcer,Techno Union,49000,10.96,100,0,4,none,none,droid tank,[https://swapi.dev/api/films/6/]
37,73,Droid gunship,HMP droid gunship,"Baktoid Fleet Ordnance, Haor Chall Engineering",60000,12.3,820,0,0,0,none,airspeeder,[https://swapi.dev/api/films/6/]
38,76,AT-RT,All Terrain Recon Transport,Kuat Drive Yards,40000,3.2,90,1,0,20,1 day,walker,[https://swapi.dev/api/films/6/]


In [146]:
vehicles_df[vehicles_df['cargo_capacity']=='none']

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
5,16,TIE bomber,TIE/sa bomber,Sienar Fleet Systems,-1,7.8,850,1,0,none,2 days,space/planetary bomber,"[https://swapi.dev/api/films/2/, https://swapi..."
16,36,Single Trooper Aerial Platform,Single Trooper Aerial Platform,Baktoid Armor Workshop,2500,2,400,1,0,none,none,repulsorcraft,[https://swapi.dev/api/films/4/]
36,72,Corporate Alliance tank droid,NR-N99 Persuader-class droid enforcer,Techno Union,49000,10.96,100,0,4,none,none,droid tank,[https://swapi.dev/api/films/6/]


In [147]:
#We give it the none values the value of -1.
vehicles_df['cargo_capacity'].replace(to_replace='none',value=-1,inplace=True)
vehicles_df.tail()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
34,70,Raddaugh Gnasp fluttercraft,Raddaugh Gnasp fluttercraft,Appazanna Engineering Works,14750,7,310,2,0,20,none,air speeder,[https://swapi.dev/api/films/6/]
35,71,Clone turbo tank,HAVw A6 Juggernaut,Kuat Drive Yards,350000,49.4,160,20,300,30000,20 days,wheeled walker,[https://swapi.dev/api/films/6/]
36,72,Corporate Alliance tank droid,NR-N99 Persuader-class droid enforcer,Techno Union,49000,10.96,100,0,4,-1,none,droid tank,[https://swapi.dev/api/films/6/]
37,73,Droid gunship,HMP droid gunship,"Baktoid Fleet Ordnance, Haor Chall Engineering",60000,12.3,820,0,0,0,none,airspeeder,[https://swapi.dev/api/films/6/]
38,76,AT-RT,All Terrain Recon Transport,Kuat Drive Yards,40000,3.2,90,1,0,20,1 day,walker,[https://swapi.dev/api/films/6/]


In [148]:
#We tranform the numeric columns
vehicles_df = vehicles_df.astype({'cost_in_credits':'int64','max_atmosphering_speed':'int64',
                                    'passengers':'int64','cargo_capacity':'int64'})
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Veh_Id                  39 non-null     int64 
 1   name                    39 non-null     object
 2   model                   39 non-null     object
 3   manufacturer            39 non-null     object
 4   cost_in_credits         39 non-null     int64 
 5   length                  39 non-null     object
 6   max_atmosphering_speed  39 non-null     int64 
 7   crew                    39 non-null     object
 8   passengers              39 non-null     int64 
 9   cargo_capacity          39 non-null     int64 
 10  consumables             39 non-null     object
 11  vehicle_class           39 non-null     object
 12  films                   39 non-null     object
dtypes: int64(5), object(8)
memory usage: 4.1+ KB


In [149]:
'''We could transform the values that we input to the unknown values into null values but this is going to make that the dtype
transform into float64. Here is an example for this particular table.'''
import numpy as np
vehicles_df['cost_in_credits'].replace(to_replace=-1,value=np.NaN,inplace=True)
vehicles_df['cargo_capacity'].replace(to_replace=-1,value=np.NaN,inplace=True)
vehicles_df['passengers'].replace(to_replace=-1,value=np.NaN,inplace=True)
vehicles_df['max_atmosphering_speed'].replace(to_replace=0,value=np.NaN,inplace=True)
vehicles_df['length'].replace(to_replace=0,value=np.NaN,inplace=True)
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Veh_Id                  39 non-null     int64  
 1   name                    39 non-null     object 
 2   model                   39 non-null     object 
 3   manufacturer            39 non-null     object 
 4   cost_in_credits         21 non-null     float64
 5   length                  38 non-null     object 
 6   max_atmosphering_speed  38 non-null     float64
 7   crew                    39 non-null     object 
 8   passengers              38 non-null     float64
 9   cargo_capacity          31 non-null     float64
 10  consumables             39 non-null     object 
 11  vehicle_class           39 non-null     object 
 12  films                   39 non-null     object 
dtypes: float64(4), int64(1), object(8)
memory usage: 4.1+ KB


In [150]:
vehicles_df.head()

,Veh_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,films
0,4,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000.0,36.8,30.0,46,30.0,50000.0,2 months,wheeled,"[https://swapi.dev/api/films/1/, https://swapi..."
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500.0,10.4,1200.0,1,1.0,50.0,0,repulsorcraft,[https://swapi.dev/api/films/1/]
2,7,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550.0,3.4,250.0,1,1.0,5.0,unknown,repulsorcraft,[https://swapi.dev/api/films/1/]
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NaN,6.4,1200.0,1,0.0,65.0,2 days,starfighter,"[https://swapi.dev/api/films/1/, https://swapi..."
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,NaN,4.5,650.0,2,0.0,10.0,none,airspeeder,[https://swapi.dev/api/films/2/]


In [151]:
#We are not going to do it with the other tables. It was just to show an alternative. We continue with starships.
starships_df.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/


In [152]:
starships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    36 non-null     object
 1   model                   36 non-null     object
 2   manufacturer            36 non-null     object
 3   cost_in_credits         36 non-null     object
 4   length                  36 non-null     object
 5   max_atmosphering_speed  36 non-null     object
 6   crew                    36 non-null     object
 7   passengers              36 non-null     object
 8   cargo_capacity          36 non-null     object
 9   consumables             36 non-null     object
 10  hyperdrive_rating       36 non-null     object
 11  MGLT                    36 non-null     object
 12  starship_class          36 non-null     object
 13  pilots                  36 non-null     object
 14  films                   36 non-null     object
 15  created 

In [153]:
#We are going to do the same that we have done on the other tables. Give it an Id.
starships_df['url'][0].index('2')

32

In [154]:
stsh_ids = []
for i in range(starships_df.shape[0]):
    stsh_ids.append(int(starships_df['url'][i][32:-1]))
stsh_ids

[2,
 3,
 5,
 9,
 10,
 11,
 12,
 13,
 15,
 17,
 21,
 22,
 23,
 27,
 28,
 29,
 31,
 32,
 39,
 40,
 41,
 43,
 47,
 48,
 49,
 52,
 58,
 59,
 61,
 63,
 64,
 65,
 66,
 68,
 74,
 75]

In [155]:
starships_df.insert(loc=0,column="Starship_Id",value=stsh_ids)
starships_df.head()

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,2,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,3,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,5,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/
3,9,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/
4,10,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/


In [156]:
#We take a look into pilots
starships_df['pilots']

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4     [https://swapi.dev/api/people/13/, https://swa...
5                                                    []
6     [https://swapi.dev/api/people/1/, https://swap...
7                     [https://swapi.dev/api/people/4/]
8                                                    []
9                                                    []
10                   [https://swapi.dev/api/people/22/]
11    [https://swapi.dev/api/people/1/, https://swap...
12                                                   []
13                                                   []
14                   [https://swapi.dev/api/people/29/]
15                                                   []
16                                                   []
17                                              

In [157]:
people_df['starships'].value_counts()

[]                                                                                                                                                                                           63
[https://swapi.dev/api/starships/12/]                                                                                                                                                         3
[https://swapi.dev/api/starships/10/, https://swapi.dev/api/starships/22/]                                                                                                                    2
[https://swapi.dev/api/starships/10/]                                                                                                                                                         2
[https://swapi.dev/api/starships/12/, https://swapi.dev/api/starships/22/]                                                                                                                    1
[https://swapi.dev/api/starships/13/]   

In [158]:
#It is a relation many to many. We could do the same as we did with vehicles but in order to simplify we are not going to.
#We are going to drop the usual columns.
starships_df.drop(columns=['url','created','edited'],inplace=True)
starships_df.head()

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
0,2,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi..."
1,3,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi..."
2,5,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/]
3,9,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/]
4,10,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi..."


In [159]:
starships_df['cost_in_credits'].value_counts()

unknown          10
240000            2
3500000           1
55000000          1
168000            1
57000000          1
155000            1
320000            1
59000000          1
1000000           1
125000000         1
35700             1
180000            1
2000000           1
200000            1
150000000         1
220000            1
175000            1
104000000         1
8500000           1
1143350000        1
149999            1
134999            1
100000            1
1000000000000     1
102500            1
Name: cost_in_credits, dtype: int64

In [ ]:
#All the unknown values of this table are going to be -1.
starships_df[starships_df['cost_in_credits']=='unknown']

In [ ]:
starships_df['cost_in_credits'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['cost_in_credits'].value_counts()

In [195]:
starships_df['length'].value_counts()

150       1
1,600     1
26.5      1
39        1
390       1
8         1
47.9      1
752       1
15.2      1
1088      1
18.5      1
1137      1
29.2      1
5.47      1
14.5      1
825       1
6.71      1
76        1
11        1
3170      1
19000     1
38        1
120000    1
34.37     1
14        1
12.5      1
9.2       1
90        1
115       1
21.5      1
20        1
300       1
1200      1
9.6       1
16.9      1
7.9       1
Name: length, dtype: int64

In [196]:
#We just need to take the comma out as thousand separator.
for i in range(starships_df.shape[0]):
    if type(starships_df['length'][i]) == str:
        starships_df['length'].replace(to_replace=starships_df['length'][i],value=starships_df['length'][i].replace(',',''),inplace=True)
starships_df['length'].value_counts()

150       1
1600      1
26.5      1
39        1
390       1
8         1
47.9      1
752       1
15.2      1
1088      1
18.5      1
1137      1
29.2      1
5.47      1
14.5      1
825       1
6.71      1
76        1
11        1
3170      1
19000     1
38        1
120000    1
34.37     1
14        1
12.5      1
9.2       1
90        1
115       1
21.5      1
20        1
300       1
1200      1
9.6       1
16.9      1
7.9       1
Name: length, dtype: int64

In [198]:
starships_df['max_atmosphering_speed'].value_counts()

-1        7
1050      5
1000      3
950       2
1100      2
2000      2
975       2
1600      1
8000      1
1150      1
1180      1
920       1
1300      1
900       1
800       1
850       1
650       1
1200      1
1000km    1
1500      1
Name: max_atmosphering_speed, dtype: int64

In [164]:
starships_df[starships_df['max_atmosphering_speed']=='unknown']

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
22,47,AA-9 Coruscant freighter,Botajef AA-9 Freighter-Liner,Botajef Shipyards,-1,390,unknown,unknown,30000,unknown,unknown,unknown,unknown,freighter,[],[https://swapi.dev/api/films/5/]
25,52,Republic Assault ship,Acclamator I-class assault ship,Rothana Heavy Engineering,-1,752,unknown,700,16000,11250000,2 years,0.6,unknown,assault ship,[],[https://swapi.dev/api/films/5/]
33,68,Banking clan frigte,Munificent-class star frigate,"Hoersch-Kessel Drive, Inc, Gwori Revolutionary...",57000000,825,unknown,200,unknown,40000000,2 years,1.0,unknown,cruiser,[],[https://swapi.dev/api/films/6/]


In [165]:
starships_df[starships_df['max_atmosphering_speed']=='n/a']

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
3,9,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/]
8,15,Executor,Executor-class star dreadnought,"Kuat Drive Yards, Fondor Shipyards",1143350000,19000,n/a,"279,144",38000,250000000,6 years,2.0,40,Star dreadnought,[],"[https://swapi.dev/api/films/2/, https://swapi..."
13,27,Calamari Cruiser,MC80 Liberty type Star Cruiser,Mon Calamari shipyards,104000000,1200,n/a,5400,1200,unknown,2 years,1.0,60,Star Cruiser,[],[https://swapi.dev/api/films/3/]
17,32,Droid control ship,Lucrehulk-class Droid Control Ship,"Hoersch-Kessel Drive, Inc.",-1,3170,n/a,175,139000,4000000000,500 days,2.0,unknown,Droid control ship,[],"[https://swapi.dev/api/films/4/, https://swapi..."


In [ ]:
#We replace unlnown and n/a for -1 and we should take out km
starships_df['max_atmosphering_speed'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['max_atmosphering_speed'].replace(to_replace='n/a',value=-1,inplace=True)
for i in range(starships_df.shape[0]):
    if type(starships_df['max_atmosphering_speed'][i]) == str:
        starships_df['max_atmosphering_speed'].replace(to_replace=starships_df['max_atmosphering_speed'][i],
                                            value=starships_df['max_atmosphering_speed'][i].replace('km',''),inplace=True)
starships_df['max_atmosphering_speed'].value_counts()

In [167]:
starships_df['crew'].value_counts()

1          11
5           3
3           3
4           2
6           2
30-165      1
175         1
7400        1
600         1
700         1
unknown     1
8           1
5400        1
9           1
47,060      1
854         1
279,144     1
2           1
342,953     1
200         1
Name: crew, dtype: int64

In [168]:
starships_df[starships_df['crew']=='unknown']

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
22,47,AA-9 Coruscant freighter,Botajef AA-9 Freighter-Liner,Botajef Shipyards,-1,390,-1,unknown,30000,unknown,unknown,unknown,unknown,freighter,[],[https://swapi.dev/api/films/5/]


In [169]:
starships_df[starships_df['crew']=='30-165']

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
0,2,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi..."


In [ ]:
#We are going to leave 165 for this register. So it is the max crew. We replace unknown for -1 and take out the commas.
starships_df['crew'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['crew'].replace(to_replace='30-165',value=165,inplace=True)

In [ ]:
for i in range(starships_df.shape[0]):
    if type(starships_df['crew'][i]) == str:
        starships_df['crew'].replace(to_replace=starships_df['crew'][i],value=starships_df['crew'][i].replace(',',''),inplace=True)
starships_df['crew'].value_counts()

In [ ]:
starships_df['passengers'].value_counts()

In [ ]:
starships_df['passengers'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['passengers'].replace(to_replace='n/a',value=-1,inplace=True)
for i in range(starships_df.shape[0]):
    if type(starships_df['passengers'][i]) == str:
        starships_df['passengers'].replace(to_replace=starships_df['passengers'][i],
                                            value=starships_df['passengers'][i].replace(',',''),inplace=True)
starships_df['passengers'].value_counts()

In [ ]:
starships_df['cargo_capacity'].value_counts()

In [ ]:
starships_df['cargo_capacity'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['cargo_capacity'].value_counts()

In [190]:
starships_df['hyperdrive_rating'].value_counts()

1.0        15
2.0         7
1.5         3
4.0         2
0.5         2
3.0         1
1.8         1
0.7         1
unknown     1
0.9         1
0.6         1
6           1
Name: hyperdrive_rating, dtype: int64

In [191]:
#Because this is a float number we can replace it with np.Nan instead of -1
starships_df['hyperdrive_rating'].replace(to_replace='unknown',value=np.NaN,inplace=True)
starships_df['hyperdrive_rating'].value_counts()

1.0    15
2.0     7
1.5     3
4.0     2
0.5     2
3.0     1
1.8     1
0.7     1
0.9     1
0.6     1
6       1
Name: hyperdrive_rating, dtype: int64

In [192]:
starships_df['MGLT'].value_counts()

unknown    19
60          3
70          2
100         2
40          2
10          1
75          1
80          1
105         1
20          1
50          1
120         1
91          1
Name: MGLT, dtype: int64

In [193]:
starships_df['MGLT'].replace(to_replace='unknown',value=-1,inplace=True)
starships_df['MGLT'].value_counts()

-1     19
60      3
70      2
100     2
40      2
10      1
75      1
80      1
105     1
20      1
50      1
120     1
91      1
Name: MGLT, dtype: int64

In [200]:
starships_df = starships_df.astype({'cost_in_credits':'int64','length':'float64','max_atmosphering_speed':'int64',
                                    'crew':'int64','passengers':'int64','cargo_capacity':'int64',
                                    'hyperdrive_rating':'float64','MGLT':'int64'})
starships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Starship_Id             36 non-null     int64  
 1   name                    36 non-null     object 
 2   model                   36 non-null     object 
 3   manufacturer            36 non-null     object 
 4   cost_in_credits         36 non-null     int64  
 5   length                  36 non-null     float64
 6   max_atmosphering_speed  36 non-null     int64  
 7   crew                    36 non-null     int64  
 8   passengers              36 non-null     int64  
 9   cargo_capacity          36 non-null     int64  
 10  consumables             36 non-null     object 
 11  hyperdrive_rating       35 non-null     float64
 12  MGLT                    36 non-null     int64  
 13  starship_class          36 non-null     object 
 14  pilots                  36 non-null     obje

In [201]:
starships_df.head()

,Starship_Id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
0,2,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150.00,950,165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi..."
1,3,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,1600.00,975,47060,-1,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi..."
2,5,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38.00,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/]
3,9,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000.00,-1,342953,843342,1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/]
4,10,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi..."
